In [1]:
import pandas as pd  
import numpy as np  
import matplotlib.pyplot as plt 
from datetime import timedelta
%matplotlib inline

#from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score

import xgboost as xgb
from sklearn.metrics import mean_squared_error

In [2]:
labels = pd.read_csv("../../data/labels_training_set.csv")
df = pd.read_csv("../../data/events_up_to_01062018.csv")

C:\Users\seba\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (3,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_persons = pd.read_csv("../../data/trocafone_kaggle_test.csv")

In [4]:
df["timestamp"] = pd.to_datetime(df["timestamp"])
df["month"] = df["timestamp"].dt.month
df["day"] = df["timestamp"].dt.day

In [5]:
todas_las_personas = df[["person"]].drop_duplicates(subset="person")

In [ ]:
df.columns

In [ ]:
df["event"].value_counts()

# Pruebo tsfresh.. no mirar por ahora esta parte

In [496]:
from tsfresh import extract_relevant_features


In [509]:
df_para_probar = pd.merge(df, labels, on="person", how="inner")

In [510]:
from tsfresh import extract_features
from tsfresh import select_features
from tsfresh.utilities.dataframe_functions import impute

extracted_features = extract_features(df_para_probar, column_id="person", column_sort="timestamp", column_value="label")



Feature Extraction:   0%|                                                                       | 0/10 [00:00<?, ?it/s]

Feature Extraction:  10%|██████▏                                                       | 1/10 [05:08<46:12, 308.10s/it]

Feature Extraction:  20%|████████████▍                                                 | 2/10 [05:50<30:26, 228.35s/it]

Feature Extraction:  30%|██████████████████▌                                           | 3/10 [10:17<27:59, 239.88s/it]

Feature Extraction:  40%|████████████████████████▊                                     | 4/10 [11:18<18:37, 186.21s/it]

Feature Extraction:  50%|███████████████████████████████                               | 5/10 [15:33<17:14, 206.84s/it]

Feature Extraction:  60%|█████████████████████████████████████▏                        | 6/10 [16:07<10:20, 155.12s/it]

Feature Extraction:  70%|███████████████████████████████████████████▍                  | 7/10 [20:56<09:45, 195.33s/it]

Feature Extraction:  80%|█████

In [512]:
extracted_features.head()

variable,label__abs_energy,label__absolute_sum_of_changes,"label__agg_autocorrelation__f_agg_""mean""","label__agg_autocorrelation__f_agg_""median""","label__agg_autocorrelation__f_agg_""var""","label__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""intercept""","label__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""rvalue""","label__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""slope""","label__agg_linear_trend__f_agg_""max""__chunk_len_10__attr_""stderr""","label__agg_linear_trend__f_agg_""max""__chunk_len_50__attr_""intercept""",...,label__time_reversal_asymmetry_statistic__lag_1,label__time_reversal_asymmetry_statistic__lag_2,label__time_reversal_asymmetry_statistic__lag_3,label__value_count__value_-inf,label__value_count__value_0,label__value_count__value_1,label__value_count__value_inf,label__value_count__value_nan,label__variance,label__variance_larger_than_standard_deviation
id,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0
000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,17.0,0.0,0.0,0.0,0.0,0.0
001802e4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,19.0,0.0,0.0,0.0,0.0,0.0
0019e639,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,471.0,0.0,0.0,0.0,0.0,0.0
001b0bf9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,0.0


In [520]:
y = labels.set_index("person")["label"]
y.head()

person
0566e9c1    0
6ec7ee77    0
abe7a2fb    0
34728364    0
87ed62de    0
Name: label, dtype: int64

In [521]:
impute(extracted_features)
features_filtered = select_features(extracted_features, y)

In [522]:
features_filtered

variable,label__abs_energy,"label__fft_coefficient__coeff_4__attr_""real""","label__fft_coefficient__coeff_50__attr_""abs""","label__fft_coefficient__coeff_51__attr_""abs""",label__quantile__q_0.9,"label__fft_coefficient__coeff_52__attr_""abs""","label__fft_coefficient__coeff_53__attr_""abs""","label__fft_coefficient__coeff_54__attr_""abs""","label__fft_coefficient__coeff_55__attr_""abs""","label__fft_coefficient__coeff_56__attr_""abs""",...,label__percentage_of_reoccurring_values_to_all_values,"label__fft_coefficient__coeff_45__attr_""angle""","label__fft_coefficient__coeff_66__attr_""angle""","label__fft_coefficient__coeff_85__attr_""imag""","label__fft_coefficient__coeff_71__attr_""angle""","label__fft_coefficient__coeff_92__attr_""imag""","label__fft_coefficient__coeff_53__attr_""imag""","label__fft_coefficient__coeff_58__attr_""imag""","label__fft_coefficient__coeff_89__attr_""angle""","label__fft_coefficient__coeff_69__attr_""imag"""
id,,,,,,,,,,,,,,,,,,,,,
0008ed71,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
000c79fe,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001802e4,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0019e639,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,180.0,0.0,-0.0,0.0,0.0,0.0,-0.0,0.0
001b0bf9,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001ca5ee,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001dfc31,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
001e9aea,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0020152e,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [501]:
df_para_probar = pd.merge(df, labels, on="person", how="inner").drop(columns="label")
features_filtered_direct = extract_relevant_features(df_para_probar, labels,
                                                     column_id='person', column_sort='timestamp')

ValueError: You have NaN values in your value column.

# Cantidad de veces que realiza un evento dado

In [192]:
cant_por_evento = pd.concat([pd.get_dummies(df['event']),df[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

df_train = pd.merge(cant_por_evento, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product', 'visited site', 'staticpage', 'lead'])
df_train.head()

,ad campaign hit,brand listing,checkout,conversion,generic listing,search engine hit,searched products,label
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,9.0,0
2,5.0,0.0,1.0,0.0,4.0,0.0,4.0,0
3,29.0,165.0,15.0,2.0,28.0,13.0,11.0,0
4,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0


In [193]:
#veo cuantos users pierdo
cant_por_evento_predecir = pd.merge(cant_por_evento, df_persons, on="person", how="inner")
cant_por_evento_predecir["person"].count() - df_persons["person"].count()

0

In [194]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [195]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [196]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [197]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 10, min_child_weight=1)


In [198]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=10, seed=None,
       silent=True, subsample=1)

In [199]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [200]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8537763183310798
Test acuracy:  0.8503734226113829
ROC auc score:  0.8260601668812715
Confusion matrix: 
[[3198  494]
 [  87  104]]


# Cantidad de veces que realiza un evento dado en el mes 5

In [212]:
eventos_mes_cinco = df.loc[df["month"] == 5]
cant_por_evento_mes_cinco = pd.concat([pd.get_dummies(eventos_mes_cinco['event']),eventos_mes_cinco[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()

cant_por_evento_mes_cinco.columns = ['person', 'ad campaign hit mes 5', 'brand listing mes 5', 'checkout mes 5',\
                                     'conversion mes 5', 'generic listing mes 5', 'lead mes 5', 'search engine hit mes 5',\
                                     'searched products mes 5', 'staticpage mes 5', 'viewed product mes 5', 'visited site mes 5' ]

cant_por_evento_mes_cinco = pd.merge(cant_por_evento_mes_cinco, todas_las_personas, on="person", how='right')

df_train = pd.merge(cant_por_evento_mes_cinco, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product mes 5', 'visited site mes 5', 'staticpage mes 5', 'lead mes 5'])
cant_por_evento_mes_cinco.head()

,person,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,lead mes 5,search engine hit mes 5,searched products mes 5,staticpage mes 5,viewed product mes 5,visited site mes 5
0,0008ed71,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,2.0
1,00091926,15.0,25.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,372.0,34.0
2,000ba417,1.0,24.0,6.0,1.0,14.0,0.0,1.0,0.0,0.0,153.0,6.0
3,000c79fe,1.0,0.0,1.0,0.0,1.0,0.0,1.0,9.0,0.0,3.0,1.0
4,000e4d9e,19.0,17.0,1.0,0.0,17.0,0.0,5.0,0.0,0.0,339.0,13.0


In [314]:
df_combino = pd.merge(cant_por_evento_mes_cinco, cant_por_evento, on='person', how='inner')

df_train = pd.merge(df_combino, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [315]:
#veo cuantos users pierdo
cant_por_evento_predecir = pd.merge(cant_por_evento_mes_cinco, df_persons, on="person", how="inner")
cant_por_evento_predecir["person"].count() - df_persons["person"].count()

0

In [316]:
df_train.head()

,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,search engine hit mes 5,searched products mes 5,viewed product mes 5,visited site mes 5,ad campaign hit,brand listing,checkout,conversion,generic listing,search engine hit,searched products,label
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,9.0,3.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,9.0,0
2,5.0,0.0,1.0,0.0,4.0,0.0,4.0,4.0,1.0,5.0,0.0,1.0,0.0,4.0,0.0,4.0,0
3,16.0,113.0,4.0,1.0,13.0,7.0,1.0,123.0,12.0,29.0,165.0,15.0,2.0,28.0,13.0,11.0,0
4,0.0,1.0,2.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0


In [317]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [318]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [319]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [360]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [361]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [362]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [363]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9083124074431782
Test acuracy:  0.8956992016482102
ROC auc score:  0.8517455322306225
Confusion matrix: 
[[3391  279]
 [ 126   87]]


#### Cantidad de eventos en un dia del mes

In [364]:
df_day = pd.concat([df['person'],pd.get_dummies(df['day'])],axis = 1).groupby('person').sum().\
    reset_index()

In [365]:
columnas = ['dia_' + str(i) for i in range(1,32)]
columnas.insert(0,'person')
df_day.columns = columnas

In [366]:
df_con_dias_del_mes = pd.merge(df_combino, df_day, on='person', how='inner')

df_train = pd.merge(df_con_dias_del_mes, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [367]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [368]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [369]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [370]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [371]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [372]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [373]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9057369132702338
Test acuracy:  0.8969868658253928
ROC auc score:  0.8561512581392077
Confusion matrix: 
[[3395  275]
 [ 125   88]]


#### Cantidad de eventos en top K semanas del año.

In [431]:
K = 10
df['week_of_year'] = df['timestamp'].dt.weekofyear

In [432]:
df_week_of_year = df[['person','week_of_year']]

top_K_week_of_year = df_week_of_year['week_of_year'].value_counts().head(K).index

df_week_of_year = pd.concat([df_week_of_year,pd.get_dummies(df_week_of_year['week_of_year'])[top_K_week_of_year]],axis = 1)

df_week_of_year.drop(columns = ['week_of_year'],inplace = True)

df_week_of_year1 = df_week_of_year.groupby('person').sum().reset_index()

In [433]:
df_con_top_semanas = pd.merge(df_con_dias_del_mes, df_week_of_year1, on='person', how='inner')

df_train = pd.merge(df_con_top_semanas, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [444]:
df_train.head()

,ad campaign hit mes 5,brand listing mes 5,checkout mes 5,conversion mes 5,generic listing mes 5,search engine hit mes 5,searched products mes 5,viewed product mes 5,visited site mes 5,ad campaign hit,...,20,22,19,18,17,16,15,14,13,label
0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,...,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1.0,0.0,1.0,0.0,1.0,1.0,9.0,3.0,1.0,1.0,...,0.0,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,5.0,0.0,1.0,0.0,4.0,0.0,4.0,4.0,1.0,5.0,...,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,16.0,113.0,4.0,1.0,13.0,7.0,1.0,123.0,12.0,29.0,...,181.0,0.0,0.0,108.0,0.0,0.0,0.0,0.0,0.0,0
4,0.0,1.0,2.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [535]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [536]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [537]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [538]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=17)


In [539]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [540]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [541]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9067671109394115
Test acuracy:  0.8954416688127737
ROC auc score:  0.8574055596065037
Confusion matrix: 
[[3389  281]
 [ 125   88]]


In [544]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.9067671109394115
Test acuracy:  0.8954416688127737

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      3670
           1       0.24      0.41      0.30       213

   micro avg       0.90      0.90      0.90      3883
   macro avg       0.60      0.67      0.62      3883
weighted avg       0.92      0.90      0.91      3883


Precision Score:  0.23848238482384823
Recall Score:  0.4131455399061033
F1 Score:  0.3024054982817869
Cohen Kappa Score:  0.25025490889860336
ROC auc score:  0.8574055596065037
Confusion matrix: 
[[3389  281]
 [ 125   88]]


# Usando random oversampling

In [516]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1].fillna(0), df_train.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [517]:
cant_ceros = np.sum(y_train == 0)
cant_unos = np.sum(y_train == 1)
cant_ceros, cant_unos

(14764, 767)

In [518]:
np.any(np.isnan(X_train))

False

In [519]:
np.all(np.isfinite(X))

True

In [520]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 14764, 1: 1200})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [529]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6,eval_metric = "auc", scale_pos_weight=5, min_child_weight=1)

In [530]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=True, subsample=1)

In [531]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [533]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8484715610122776
Test acuracy:  0.8411022405356683
ROC auc score:  0.8582332322728377
Confusion matrix: 
[[3127  543]
 [  74  139]]


In [534]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8484715610122776
Test acuracy:  0.8411022405356683

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.85      0.91      3670
           1       0.20      0.65      0.31       213

   micro avg       0.84      0.84      0.84      3883
   macro avg       0.59      0.75      0.61      3883
weighted avg       0.93      0.84      0.88      3883


Precision Score:  0.20381231671554254
Recall Score:  0.6525821596244131
F1 Score:  0.3106145251396648
Cohen Kappa Score:  0.24772470580921035
ROC auc score:  0.8582332322728377
Confusion matrix: 
[[3127  543]
 [  74  139]]


# Entradas en el mes cinco

In [170]:
entradas_mes_5 = df.loc[df["month"] == 5]
cant_entradas_mes_5 = entradas_mes_5.groupby("person").agg({"month": "count"}).reset_index()
cant_entradas_mes_5.columns = ["person", "cant_mes_5"]
cant_entradas_mes_5 = pd.merge(cant_entradas_mes_5, todas_las_personas, on="person", how="right").fillna(0)
cant_entradas_mes_5.head()

,person,cant_mes_5
0,0008ed71,6.0
1,00091926,448.0
2,000ba417,206.0
3,000c79fe,17.0
4,000e4d9e,411.0


In [171]:
# Merge

df_con_mes_5 = pd.merge(cant_entradas_mes_5, cant_por_evento, on="person", how="inner")
df_train = pd.merge(df_con_mes_5, labels, on="person", how="inner")\
            .drop(columns = ['person', 'viewed product', 'visited site', 'staticpage', 'lead'])

In [172]:
# Veo cuantas personas a predecir pierdo
entradas_mes_5_predecir = pd.merge(cant_entradas_mes_5, df_persons, on="person", how="inner")
entradas_mes_5_predecir["person"].count() - df_persons["person"].count()

0

In [173]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [174]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [175]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [188]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 10, min_child_weight=1)


In [189]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=10, seed=None,
       silent=True, subsample=1)

In [190]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [191]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8771634453498552
Test acuracy:  0.8569889577161325
ROC auc score:  0.8233341063353461
Confusion matrix: 
[[3084  438]
 [  93   98]]


# Distancia en dias de 1er y ultimo dia de entrada al sitio

In [91]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
primer_dia = timestamp_df.groupby("person")["timestamp"].min().rename("primer").reset_index()
distancia_dias = pd.merge(primer_dia, ultimo_dia, on="person", how="inner")
distancia_dias["distan_dias"] = (distancia_dias["ultimo"] - distancia_dias["primer"]).dt.days
distancia_dias = distancia_dias[["person", "distan_dias"]]
distancia_dias.head()

,person,distan_dias
0,0008ed71,0
1,00091926,27
2,00091a7a,0
3,000ba417,9
4,000c79fe,0


In [596]:
# Merge
#df_con_mes_5
df_con_distancias = pd.merge(distancia_dias, df_con_top_semanas, on="person", how="inner") 
df_train = pd.merge(df_con_distancias, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [597]:
# Veo cuantas personas a predecir pierdo
distancia_dias_predecir = pd.merge(distancia_dias, df_persons, on="person", how="inner")
distancia_dias_predecir["person"].count() - df_persons["person"].count()

0

In [598]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [599]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [600]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [605]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=17)


In [606]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [607]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [608]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8548065160002576
Test acuracy:  0.8532062838011847
ROC auc score:  0.8468962182276099
Confusion matrix: 
[[3196  496]
 [  74  117]]


## Cantidad de modelos distintos vistos por cada usuario

In [545]:
modelos_df = df[["person", "model"]]
cant_modelos = modelos_df.groupby("person")["model"].nunique().rename("modelos_dist").reset_index()
cant_modelos.head()

,person,modelos_dist
0,0008ed71,3
1,00091926,36
2,00091a7a,3
3,000ba417,26
4,000c79fe,1


In [609]:
# Merge 
#df_con_distancias
df_con_modelos = pd.merge(df_con_top_semanas, cant_modelos, on="person", how="inner") 
df_train = pd.merge(df_con_modelos, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

In [610]:
# Veo cuantas personas a predecir pierdo
cant_modelos_predecir = pd.merge(cant_modelos, df_persons, on="person", how="inner")
cant_modelos_predecir["person"].count() - df_persons["person"].count()

0

In [611]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [612]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [613]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [614]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=1)


In [615]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [616]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [617]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.882042366879145
Test acuracy:  0.8720061807880505
ROC auc score:  0.8561429430351409
Confusion matrix: 
[[3275  395]
 [ 102  111]]


## Cantidad de días distintos de entrada al sitio

In [121]:
df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()
cant_dias_dist.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,cant_dias_dist
0,0008ed71,1
1,00091926,22
2,00091a7a,1
3,000ba417,3
4,000c79fe,1


In [637]:
# Merge df_con_modelos

df_con_dias_dist = pd.merge(df_con_top_semanas, cant_dias_dist, on="person", how="inner") 
df_train = pd.merge(df_con_dias_dist, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])

df_train.columns

Index([  'ad campaign hit mes 5',     'brand listing mes 5',
                'checkout mes 5',        'conversion mes 5',
         'generic listing mes 5', 'search engine hit mes 5',
       'searched products mes 5',    'viewed product mes 5',
            'visited site mes 5',         'ad campaign hit',
                 'brand listing',                'checkout',
                    'conversion',         'generic listing',
             'search engine hit',       'searched products',
                         'dia_1',                   'dia_2',
                         'dia_3',                   'dia_4',
                         'dia_5',                   'dia_6',
                         'dia_7',                   'dia_8',
                         'dia_9',                  'dia_10',
                        'dia_11',                  'dia_12',
                        'dia_13',                  'dia_14',
                        'dia_15',                  'dia_16',
                        

In [619]:
# Veo cuantas personas a predecir pierdo
cant_dias_dist_predecir = pd.merge(cant_dias_dist, df_persons, on="person", how="inner")
cant_dias_dist_predecir["person"].count() - df_persons["person"].count()

0

In [620]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [621]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [622]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [623]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=17)


In [624]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [625]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [626]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8837164380915589
Test acuracy:  0.8689157867628122
ROC auc score:  0.854903352905809
Confusion matrix: 
[[3266  404]
 [ 105  108]]


# Entro como 'new' en el mes 5

In [653]:
df_mes_5 = df[["person", "month", "new_vs_returning"]].loc[df["month"] == 5]
df_mes_5["nuevo_mes_5"] = (df_mes_5["new_vs_returning"] == "New") * 1
df_mew_5 = df_mes_5[["person", "nuevo_mes_5"]]
df_mew_5 = df_mew_5.sort_values("nuevo_mes_5", ascending=False)
df_mew_5 = df_mew_5.drop_duplicates(subset=['person'], keep='first')
df_mew_5 = pd.merge(df_mew_5, todas_las_personas, on="person", how="right").fillna(0)
df_mew_5.head()

,person,nuevo_mes_5
0,c0f2beb1,1.0
1,9ff9d5d2,1.0
2,ae9e5fa3,1.0
3,bbbe368d,1.0
4,0b7a3edd,1.0


In [654]:
# Veo cuantas personas a predecir pierdo
df_mew_5_predecir = pd.merge(df_mew_5, df_persons, on="person", how="inner")
df_mew_5_predecir["person"].count() - df_persons["person"].count()

0

In [655]:
# Merge

df_con_new_mes5 = pd.merge(df_con_top_semanas, df_mew_5, on="person", how="inner") 
df_train = pd.merge(df_con_new_mes5, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])
df_train['nuevo_mes_5'].value_counts()

1.0    14789
0.0     4625
Name: nuevo_mes_5, dtype: int64

In [656]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [657]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [658]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [667]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=17)


In [668]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=17, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [669]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [670]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8863563196188269
Test acuracy:  0.8714911151171775
ROC auc score:  0.8565925982781337
Confusion matrix: 
[[3274  396]
 [ 103  110]]


## Visitas a la pagina segun resolucion

In [671]:
df_resolucion = df.loc[df['event'] == 'visited site']
df_resolucion = df_resolucion[['person','screen_resolution']].dropna()

top_5_resoluciones = df_resolucion['screen_resolution'].value_counts().head(10).index
df_resolucion = df_resolucion.loc[df_resolucion['screen_resolution'].isin(top_5_resoluciones)]
df_resolucion = pd.concat([df_resolucion['person'],\
        pd.get_dummies(df_resolucion['screen_resolution'])],axis = 1).groupby('person').sum().reset_index()

df_resolucion = pd.merge(df_resolucion, todas_las_personas, on="person", how="right").fillna(0)

df_resolucion.head()

,person,1024x768,1360x768,1366x768,1440x900,1600x900,1920x1080,320x534,320x568,320x570,360x640
0,0008ed71,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,00091926,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000ba417,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [672]:
# Creo que es mucho 20 resoluciones, probar (quizas empeore)..

In [673]:
# Veo cuantas personas a predecir pierdo
df_resolucion_predecir = pd.merge(df_resolucion, df_persons, on="person", how="inner")
df_resolucion_predecir["person"].count() - df_persons["person"].count()

0

In [675]:
# Merge

df_con_resoluciones = pd.merge(df_con_top_semanas, df_resolucion, on="person", how="inner") 
df_train = pd.merge(df_con_resoluciones, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


In [676]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [677]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [678]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [715]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [716]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [717]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [718]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9311055308737364
Test acuracy:  0.9175894926603142
ROC auc score:  0.8584545419656907
Confusion matrix: 
[[3500  170]
 [ 150   63]]


# Usando random oversampling

In [719]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1].fillna(0), df_train.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [720]:
cant_ceros = np.sum(y_train == 0)
cant_unos = np.sum(y_train == 1)
cant_ceros, cant_unos

(14764, 767)

In [721]:
np.any(np.isnan(X_train))

False

In [722]:
np.all(np.isfinite(X))

True

In [723]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 14764, 1: 1200})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [751]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6,eval_metric = "auc", scale_pos_weight=5, min_child_weight=20)

In [752]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=20, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=True, subsample=1)

In [753]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [754]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8737158606865447
Test acuracy:  0.871233582281741
ROC auc score:  0.860972739251129
Confusion matrix: 
[[3260  410]
 [  90  123]]


In [742]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8737158606865447
Test acuracy:  0.871233582281741

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      3670
           1       0.23      0.58      0.33       213

   micro avg       0.87      0.87      0.87      3883
   macro avg       0.60      0.73      0.63      3883
weighted avg       0.93      0.87      0.90      3883


Precision Score:  0.23076923076923078
Recall Score:  0.5774647887323944
F1 Score:  0.3297587131367293
Cohen Kappa Score:  0.2727538338215353
ROC auc score:  0.860972739251129
Confusion matrix: 
[[3260  410]
 [  90  123]]


OCHENTA Y SEIS

### Dia, mes con mayor cantidad de visitas de cada persona

In [6]:
# Dia con mas entradas y su cantidad de entradas

entradas_x_dia = df.groupby("person")['day'].value_counts().rename("cant_dia_freq").reset_index()
entradas_x_dia.columns = ['person', 'day', 'cant_dia_freq']
entradas_x_dia = entradas_x_dia.sort_values('cant_dia_freq',ascending=False)
dia_mas_entradas = entradas_x_dia.drop_duplicates(subset=['person'])
dia_mas_entradas.head()

,person,day,cant_dia_freq
99325,c76b8417,3,652
28266,37eff05b,29,616
112559,e1443dd4,30,572
51164,656c18ef,10,512
41082,50e16a8a,19,423


In [7]:
# Mes con mas entradas y su cantidad de entradas

entradas_x_mes = df.groupby("person")['month'].value_counts().rename("cant_mes").reset_index()
entradas_x_mes.columns = ['person', 'month', 'cant_mes']
entradas_x_mes = entradas_x_mes.sort_values('cant_mes',ascending=False)
mes_mas_entradas = entradas_x_mes.drop_duplicates(subset=['person'])
mes_mas_entradas.head()

,person,month,cant_mes
38966,c76b8417,5,3028
18217,5c76e694,5,1609
32890,a7ffa917,5,1604
4824,18489dd5,5,1563
9989,33385551,4,1539


In [8]:
# Merge

cant_mes_mas_entradas = mes_mas_entradas[["person", "cant_mes"]]
cant_dia_mas_entradas = dia_mas_entradas[["person", "cant_dia_freq"]]

mes_dia_mas_entradas = pd.merge(cant_mes_mas_entradas, cant_dia_mas_entradas, on="person", how="inner")

df_con_labels = pd.merge(mes_dia_mas_entradas, labels, on="person", how="inner")
df_con_labels.head()

,person,cant_mes,cant_dia_freq,label
0,5c76e694,1609,191,0
1,18489dd5,1563,174,0
2,1775ba85,1450,126,0
3,6abd2bf1,1334,300,0
4,97b0c0d1,1328,217,0


In [9]:
mes_dia_mas_entradas_predecir = pd.merge(mes_dia_mas_entradas, df_persons, on="person", how="inner")

In [10]:
mes_dia_mas_entradas_predecir["person"].count() - df_persons["person"].count()

0

In [11]:
df_con_labels_num = df_con_labels[["cant_mes", "cant_dia_freq", "label"]]

## Prueba

In [463]:
# Separamos la variable a predecir

X, y = df_con_labels_num.iloc[:,:-1],df_con_labels_num.iloc[:,-1]

In [464]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [465]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [466]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=1)


In [467]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [468]:
# Predecimos

preds = xg_reg.predict(X_test)

In [469]:
# Vemos que onda

#train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
#test_accuracy = accuracy_score(y_test, preds)
#matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9475886935805807
Test acuracy:  0.9443729075457121
ROC auc score:  0.6264300612293363
Confusion matrix: 
[[3660   12]
 [ 204    7]]


In [434]:
# Separamos la variable a predecir

X, y = df_con_labels_num.iloc[:,:-1],df_con_labels_num.iloc[:,-1]

In [435]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [436]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [457]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=1)


In [458]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [460]:
# Predecimos

probabilidades = xg_reg.predict_proba(X_test)[:, 1]
preds = xg_reg.predict(X_test)

In [462]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, probabilidades)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9475886935805807
Test acuracy:  0.9443729075457121
ROC auc score:  0.6264300612293363
Confusion matrix: 
[[3660   12]
 [ 204    7]]


# Distancia entre ultima entrada al sitio y el ultimo dia del mes 5 que tenemos

In [925]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia_de_todos = timestamp_df["timestamp"].max()
ultima_entrada = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
ultima_entrada["dias_hasta_ultimo"] = (ultimo_dia_de_todos - ultima_entrada["ultimo"]).dt.days
ultima_entrada = ultima_entrada[["person", "dias_hasta_ultimo"]]
ultima_entrada.head()

,person,dias_hasta_ultimo
0,0008ed71,14
1,00091926,0
2,00091a7a,66
3,000ba417,5
4,000c79fe,2


In [926]:
# Veo cuantas personas a predecir pierdo
df_predecir = pd.merge(ultima_entrada, df_persons, on="person", how="inner")
df_predecir["person"].count() - df_persons["person"].count()

0

In [1051]:
# Merge
# df_con_resoluciones

df_con_diatancia_hasta_ultimo = pd.merge(df_con_resoluciones, ultima_entrada, on="person", how="inner") 
df_train = pd.merge(df_con_diatancia_hasta_ultimo, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


In [1052]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [1053]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [1054]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1055]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [1056]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [1057]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [1058]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9307835941021183
Test acuracy:  0.921452485191862
ROC auc score:  0.8602141459108877
Confusion matrix: 
[[3508  162]
 [ 143   70]]


# Usando random oversampling

In [1060]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1].fillna(0), df_train.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1061]:
cant_ceros = np.sum(y_train == 0)
cant_unos = np.sum(y_train == 1)
cant_ceros, cant_unos

(14764, 767)

In [1062]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 14764, 1: 784})
X_train_res, y_train_res = ros.fit_sample(X_train, y_train)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [1063]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6,eval_metric = "auc", scale_pos_weight=5, min_child_weight=10)

In [1064]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=5,
       seed=None, silent=True, subsample=1)

In [1065]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [1066]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.928222279392848
Test acuracy:  0.9173319598248777
ROC auc score:  0.8628461961597013
Confusion matrix: 
[[3492  178]
 [ 143   70]]


In [1067]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, proba)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.928222279392848
Test acuracy:  0.9173319598248777

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.95      0.96      3670
           1       0.28      0.33      0.30       213

   micro avg       0.92      0.92      0.92      3883
   macro avg       0.62      0.64      0.63      3883
weighted avg       0.92      0.92      0.92      3883


Precision Score:  0.28225806451612906
Recall Score:  0.3286384976525822
F1 Score:  0.3036876355748373
Cohen Kappa Score:  0.2600143076379633
ROC auc score:  0.8628461961597013
Confusion matrix: 
[[3492  178]
 [ 143   70]]


# Cantidad de eventos segun quincena

In [1110]:
df_mes_cinco = df.loc[df["month"] == 5]
df_quincenas_mes_cinco = df_mes_cinco[['person','day']]

df_quincenas_mes_cinco['primer_quincena'] = ((df_quincenas_mes_cinco['day'] >=1) & (df_quincenas_mes_cinco['day'] < 16)).astype(int)
df_quincenas_mes_cinco['segunda_quincena'] = (df_quincenas_mes_cinco['day'] > 16).astype(int)

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [1111]:
df_quincenas_mes_cinco = df_quincenas_mes_cinco.drop(columns = ['day'])

df_quincenas_mes_cinco = df_quincenas_mes_cinco.groupby('person').sum().reset_index()

# Pongo las personas que no entraron en el mes 5 en 0
df_quincenas_mes_cinco = pd.merge(df_quincenas_mes_cinco, todas_las_personas, on='person', how='right').fillna(0)
df_quincenas_mes_cinco.head()

,person,primer_quincena,segunda_quincena
0,0008ed71,0.0,6.0
1,00091926,187.0,260.0
2,000ba417,0.0,206.0
3,000c79fe,0.0,17.0
4,000e4d9e,0.0,216.0


In [1121]:
df_con_quincenas = pd.merge(df_con_diatancia_hasta_ultimo, df_quincenas_mes_cinco, on="person", how="inner") 
df_train = pd.merge(df_con_quincenas, labels, on="person", how="inner")\
            .drop(columns = ['person','viewed product', 'visited site', 'staticpage', 'lead', 'viewed product',\
                             'visited site', 'staticpage mes 5', 'lead mes 5'])


In [1122]:
# Separamos la variable a predecir

X, y = df_train.iloc[:,:-1],df_train.iloc[:,-1]

In [1123]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [1124]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1125]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 5, min_child_weight=12)


In [1126]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=12, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=5, seed=None,
       silent=True, subsample=1)

In [1127]:
# Predecimos

preds = xg_reg.predict(X_test)
proba = xg_reg.predict_proba(X_test)[:,1]

In [1128]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, proba)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9136565578520378
Test acuracy:  0.9026525882049962
ROC auc score:  0.8549295774647887
Confusion matrix: 
[[3426  244]
 [ 134   79]]


### Entradas en los ultimos 30 dias (hasta la ultima visita)

In [59]:
df_meses = df[["timestamp", "person", "month"]]
df_ultimo_dia = df_meses.groupby("person")["timestamp"].max().reset_index()
df_ultimo_dia.columns = ["person", "last_day"]
df_meses_ultimo_dia = pd.merge(df_ultimo_dia, df_meses, on="person", how="inner")
entradas_30_dias = df_meses_ultimo_dia.loc[((df_meses_ultimo_dia["last_day"] - timedelta(days=30)) < \
                                           df_meses_ultimo_dia["timestamp"]) & (df_meses_ultimo_dia["timestamp"]< \
                                           df_meses_ultimo_dia["last_day"])]


entradas_30_dias = pd.merge(entradas_30_dias, todas_las_personas, on="person", how="right").fillna(0)
entradas_30_dias.head()

Exception ignored in: <bound method DMatrix.__del__ of <xgboost.core.DMatrix object at 0x00000225C2401C18>>
Traceback (most recent call last):
  File "C:\Users\seba\Anaconda3\lib\site-packages\xgboost\core.py", line 482, in __del__
    if self.handle is not None:
AttributeError: 'DMatrix' object has no attribute 'handle'


,person,last_day,timestamp,month
0,0008ed71,2018-05-17 16:28:37,2018-05-17 12:27:47,5.0
1,0008ed71,2018-05-17 16:28:37,2018-05-17 13:45:00,5.0
2,0008ed71,2018-05-17 16:28:37,2018-05-17 16:22:06,5.0
3,0008ed71,2018-05-17 16:28:37,2018-05-17 13:44:59,5.0
4,0008ed71,2018-05-17 16:28:37,2018-05-17 16:21:54,5.0


In [60]:
entradas_30_dias_df = entradas_30_dias["person"].value_counts().rename("entradas_30_dias").reset_index()
entradas_30_dias_df.columns = ["person", "entradas_30_dias"]
entradas_30_dias_df.head()

,person,entradas_30_dias
0,c76b8417,3027
1,5c76e694,1608
2,a7ffa917,1603
3,18489dd5,1562
4,622b4acf,1458


In [61]:
# Merge

df_con_labels2 = pd.merge(entradas_30_dias_df, df_con_labels, on="person", how="inner")
df_con_labels2.head()

,person,entradas_30_dias,cant_mes,cant_dia_freq,label
0,5c76e694,1608,1609,191,0
1,18489dd5,1562,1563,174,0
2,6abd2bf1,1319,1334,300,0
3,97b0c0d1,1306,1328,217,0
4,595b9b50,1177,1179,377,0


In [62]:
entradas_30_dias_predecir = pd.merge(entradas_30_dias_df, df_persons, on="person", how="inner")
entradas_30_dias_predecir["person"].count() - df_persons["person"].count()

0

In [63]:
df_con_labels2_num = df_con_labels2[["entradas_30_dias", "cant_mes", "cant_dia_freq", "label"]]

## Prueba

In [64]:
# Separamos la variable a predecir

X, y = df_con_labels2_num.iloc[:,:-1], df_con_labels2_num.iloc[:,-1]

In [65]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [66]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [67]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [68]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [69]:
# Predecimos

preds = xg_reg.predict(X_test)

In [70]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8924731182795699
Test acuracy:  0.8807622971928921
ROC auc score:  0.5287829377725018
Confusion matrix: 
[[3393  293]
 [ 170   27]]


## Entradas en el ultimo mes (mes 5)

In [71]:
entradas_mes_5 = df_meses.loc[df_meses["month"] == 5]
cant_entradas_mes_5 = entradas_mes_5.groupby("person").agg({"month": "count"}).reset_index()
cant_entradas_mes_5.columns = ["person", "cant_mes_5"]
cant_entradas_mes_5 = pd.merge(cant_entradas_mes_5, todas_las_personas, on="person", how="right").fillna(0)
cant_entradas_mes_5.head()

,person,cant_mes_5
0,0008ed71,6.0
1,00091926,448.0
2,000ba417,206.0
3,000c79fe,17.0
4,000e4d9e,411.0


In [72]:
# Merge

df_con_mes_5 = pd.merge(cant_entradas_mes_5, df_con_labels2, on="person", how="inner")
df_con_mes_5.head()

,person,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0008ed71,6.0,5,6,6,0
1,000c79fe,17.0,16,17,17,0
2,001802e4,19.0,18,19,19,0
3,0019e639,290.0,358,290,72,0
4,001b0bf9,7.0,6,7,7,0


In [73]:
# Veo cuantas personas a predecir pierdo
entradas_mes_5_predecir = pd.merge(cant_entradas_mes_5, df_persons, on="person", how="inner")
entradas_mes_5_predecir["person"].count() - df_persons["person"].count()

0

## Prueba

In [74]:
# Separamos la variable a predecir
df_con_mes_5_num = df_con_mes_5.drop(columns="person")

X, y = df_con_mes_5_num.iloc[:,:-1],df_con_mes_5_num.iloc[:,-1]

In [75]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [76]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [77]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [78]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [79]:
# Predecimos

preds = xg_reg.predict(X_test)

In [80]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8840383748631768
Test acuracy:  0.8787020345094
ROC auc score:  0.5930984636246178
Confusion matrix: 
[[3354  316]
 [ 155   58]]


## Cantidad de entradas totales

In [81]:
entradas_tot = df.groupby("person").agg({"person": "count"})
#cant_entradas_tot = entradas_tot[["person", "entradas_tot"]]
entradas_tot.columns = ["cant_entradas"]
entradas_tot.reset_index(inplace=True)
entradas_tot.head()

,person,cant_entradas
0,0008ed71,6
1,00091926,448
2,00091a7a,10
3,000ba417,206
4,000c79fe,17


In [82]:
# Merge

df_con_entradas = pd.merge(entradas_tot, df_con_mes_5, on="person", how="inner")
df_con_entradas.head()

,person,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0008ed71,6,6.0,5,6,6,0
1,000c79fe,17,17.0,16,17,17,0
2,001802e4,19,19.0,18,19,19,0
3,0019e639,471,290.0,358,290,72,0
4,001b0bf9,7,7.0,6,7,7,0


In [83]:
# Veo cuantas personas a predecir pierdo
entradas_tot_predecir = pd.merge(entradas_tot, df_persons, on="person", how="inner")
entradas_tot_predecir["person"].count() - df_persons["person"].count()

0

## Prueba

In [84]:
# Separamos la variable a predecir
df_con_entradas_num = df_con_entradas.drop(columns="person")

X, y = df_con_entradas_num.iloc[:,:-1],df_con_entradas_num.iloc[:,-1]

In [85]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [86]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [87]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [88]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [89]:
# Predecimos

preds = xg_reg.predict(X_test)

In [90]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8441182151825382
Test acuracy:  0.8485706927633273
ROC auc score:  0.6473073236033194
Confusion matrix: 
[[3214  478]
 [ 110   81]]


### Cantidad de checkouts

In [91]:
checkouts = df.loc[df["event"] == "checkout"]

checkouts_tot = checkouts.groupby("person")["event"].value_counts().rename("check_tot").reset_index()
cant_checkouts_tot = checkouts_tot[["person", "check_tot"]]
cant_checkouts_tot = pd.merge(cant_entradas_mes_5, todas_las_personas, on="person", how="right").fillna(0)

In [92]:
# Merge

df_con_check = pd.merge(cant_checkouts_tot, df_con_entradas, on="person", how="inner")
df_con_check.head()

,person,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0008ed71,6.0,6,6.0,5,6,6,0
1,000c79fe,17.0,17,17.0,16,17,17,0
2,001802e4,19.0,19,19.0,18,19,19,0
3,0019e639,290.0,471,290.0,358,290,72,0
4,001b0bf9,7.0,7,7.0,6,7,7,0


In [93]:
# xgboost solo usa valores numericos
df_con_check_num = df_con_check.drop(columns="person")

In [94]:
# Veo cuantas personas a predecir pierdo
cant_checkouts_tot_predecir = pd.merge(cant_checkouts_tot, df_persons, on="person", how="inner")
cant_checkouts_tot_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [95]:
# Separamos la variable a predecir

X, y = df_con_check_num.iloc[:,:-1], df_con_check_num.iloc[:,-1]

In [96]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [97]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [98]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [99]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [100]:
# Predecimos

preds = xg_reg.predict(X_test)

In [101]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8885454896658296
Test acuracy:  0.879217100180273
ROC auc score:  0.5756815187217766
Confusion matrix: 
[[3364  306]
 [ 163   50]]


## Cantidad de entradas al evento mas visitado por cada persona

In [102]:
s = df.groupby("person")['event'].value_counts().rename().reset_index()
s.columns = ['person', 'event', 'cant_ev']
s = s.sort_values('cant_ev',ascending=False)
mayor_evento = s.drop_duplicates(subset=['person']).drop(columns="event")
mayor_evento.head()

,person,cant_ev
97161,6abd2bf1,2355
160896,b1f4dbf6,2233
129961,8fb4929e,1912
141831,9ccf882a,1891
6202,06ed04d6,1881


In [103]:
# Cantidad de entradas al modelo mas visitado por cada persona

mod = df.groupby("person")['model'].value_counts().rename().reset_index()
mod.columns = ['person', 'model', 'cant_mod']
mod = mod.sort_values('cant_mod',ascending=False)
mayor_modelo = mod.drop_duplicates(subset=['person']).drop(columns="model")
mayor_modelo = pd.merge(mayor_modelo, todas_las_personas, on="person", how="right").fillna(0)
mayor_modelo.head()

,person,cant_mod
0,b1f4dbf6,836.0
1,171e75cb,683.0
2,6f19cfd9,624.0
3,6abd2bf1,607.0
4,9bf968c5,568.0


In [104]:
# Cantidad de entradas al search engine mas visitado por cada persona

eng = df.groupby("person")['search_engine'].value_counts().rename().reset_index()
eng.columns = ['person', 'search_engine', 'cant_eng']
eng = eng.sort_values('cant_eng',ascending=False)
mayor_engine = eng.drop_duplicates(subset=['person']).drop(columns="search_engine")
mayor_engine = pd.merge(mayor_engine, todas_las_personas, on="person", how="right").fillna(0)
mayor_engine.head()

,person,cant_eng
0,c76b8417,762.0
1,6ca3126e,245.0
2,622b4acf,206.0
3,25b77cf2,161.0
4,5c76e694,136.0


In [105]:
# Cantidad de entradas al campaign source mas visitado por cada persona

src = df.groupby("person")['campaign_source'].value_counts().rename().reset_index()
src.columns = ['person', 'campaign_source', 'cant_src']
src = src.sort_values('cant_src',ascending=False)
mayor_camp = src.drop_duplicates(subset=['person']).drop(columns="campaign_source")
mayor_camp = pd.merge(mayor_camp, todas_las_personas, on="person", how="right").fillna(0)
mayor_camp.head()

,person,cant_src
0,02f14240,500.0
1,c76b8417,374.0
2,6ca3126e,335.0
3,622b4acf,282.0
4,3b2d17f6,221.0


In [106]:
# Cantidad de entradas desde la ciudad mas frecuente de cada persona

cty = df.groupby("person")['city'].value_counts().rename().reset_index()
cty.columns = ['person', 'city', 'cant_city']
src = cty.sort_values('cant_city',ascending=False)
mayor_ciudad = src.drop_duplicates(subset=['person']).drop(columns="city")
mayor_ciudad = pd.merge(mayor_ciudad, todas_las_personas, on="person", how="right").fillna(0)
mayor_ciudad.head()

,person,cant_city
0,5059f7fd,268.0
1,7ac0c607,208.0
2,67bdc946,207.0
3,ffee0f18,173.0
4,9b3b43aa,171.0


In [107]:
# Merge

from functools import reduce

# merge de todos los dfs

dfs = [mayor_ciudad, mayor_camp, mayor_engine, mayor_modelo, mayor_evento]
df_final1 = reduce(lambda left,right: pd.merge(left,right,on='person', how='inner'), dfs)
df_final = pd.merge(df_final1, df_con_check, on="person", how="inner")
df_final.head()

,person,cant_city,cant_src,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,ffee0f18,173.0,76.0,65.0,80.0,1518,218.0,3458,218.0,217,1028,282,0
1,9b3b43aa,171.0,17.0,4.0,468.0,1289,246.0,1663,246.0,234,923,174,0
2,02f14240,155.0,500.0,6.0,272.0,523,225.0,1798,225.0,282,847,114,0
3,345fc18e,153.0,155.0,10.0,106.0,844,367.0,1569,367.0,364,1007,137,0
4,30ec9daf,152.0,30.0,10.0,83.0,423,185.0,803,185.0,184,189,68,0


In [108]:
df_final_num = df_final.drop(columns="person")

In [109]:
# Veo cuantas personas a predecir pierdo
df1_predecir = pd.merge(df_final1, df_persons, on="person", how="inner")
df1_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [110]:
# Separamos la variable a predecir

X, y = df_final_num.iloc[:,:-1], df_final_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [111]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [112]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [113]:
# Predecimos

preds = xg_reg.predict(X_test)

In [114]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.7690425600412079
Test acuracy:  0.7532835436518156
ROC auc score:  0.6571651387827858
Confusion matrix: 
[[2822  874]
 [  84  103]]


In [122]:
# Separamos la variable a predecir

X, y = df_final_num.iloc[:,:-1],df_final_num.iloc[:,-1]

In [123]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [124]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [141]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 10, min_child_weight=1)


In [142]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=10, seed=None,
       silent=True, subsample=1)

In [143]:
# Predecimos

preds = xg_reg.predict_proba(X_test)[:, 1]
preds

array([0.60300773, 0.42328286, 0.3905003 , ..., 0.37030753, 0.27939197,
       0.40139404], dtype=float32)

In [144]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, xg_reg.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, xg_reg.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8640139076685339
Test acuracy:  0.8570692763327324
ROC auc score:  0.7274687767669051
Confusion matrix: 
[[3260  436]
 [ 119   68]]


## Distancia entre primer y ultimo dia de entrada al sitio

In [145]:
timestamp_df = df[["person", "timestamp"]]
ultimo_dia = timestamp_df.groupby("person")["timestamp"].max().rename("ultimo").reset_index()
primer_dia = timestamp_df.groupby("person")["timestamp"].min().rename("primer").reset_index()
distancia_dias = pd.merge(primer_dia, ultimo_dia, on="person", how="inner")
distancia_dias["distan_dias"] = (distancia_dias["ultimo"] - distancia_dias["primer"]).dt.days
distancia_dias = distancia_dias[["person", "distan_dias"]]
distancia_dias.head()

,person,distan_dias
0,0008ed71,0
1,00091926,27
2,00091a7a,0
3,000ba417,9
4,000c79fe,0


In [146]:
# Merge

df_con_distancias = pd.merge(distancia_dias, df_final, on="person", how="inner") 
df_con_distancias_num = df_con_distancias.drop(columns="person")
df_con_distancias_num.head()

,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0,2.0,0.0,0.0,1.0,3,6.0,6,6.0,5,6,6,0
1,0,1.0,1.0,1.0,4.0,9,17.0,17,17.0,16,17,17,0
2,0,1.0,5.0,0.0,3.0,5,19.0,19,19.0,18,19,19,0
3,114,12.0,17.0,12.0,63.0,189,290.0,471,290.0,358,290,72,0
4,0,0.0,0.0,0.0,5.0,2,7.0,7,7.0,6,7,7,0


In [147]:
# Veo cuantas personas a predecir pierdo
distancia_dias_predecir = pd.merge(distancia_dias, df_persons, on="person", how="inner")
distancia_dias_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [148]:
# Separamos la variable a predecir

X, y = df_con_distancias_num.iloc[:,:-1], df_con_distancias_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [149]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [150]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [151]:
# Predecimos

preds = xg_reg.predict(X_test)

In [152]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8446977013714506
Test acuracy:  0.8398145763584857
ROC auc score:  0.6253261615605838
Confusion matrix: 
[[3187  505]
 [ 117   74]]


## Cantidad de modelos distintos vistos por cada usuario

In [153]:
modelos_df = df[["person", "model"]]
cant_modelos = modelos_df.groupby("person")["model"].nunique().rename("modelos_dist").reset_index()
cant_modelos.head()

,person,modelos_dist
0,0008ed71,3
1,00091926,36
2,00091a7a,3
3,000ba417,26
4,000c79fe,1


In [154]:
# Merge 

df_con_modelos = pd.merge(cant_modelos, df_con_distancias, on="person", how="inner")
df_con_modelos_num = df_con_modelos.drop(columns="person")
df_con_modelos_num.head()

,modelos_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,3,0,2.0,0.0,0.0,1.0,3,6.0,6,6.0,5,6,6,0
1,1,0,1.0,1.0,1.0,4.0,9,17.0,17,17.0,16,17,17,0
2,2,0,1.0,5.0,0.0,3.0,5,19.0,19,19.0,18,19,19,0
3,26,114,12.0,17.0,12.0,63.0,189,290.0,471,290.0,358,290,72,0
4,1,0,0.0,0.0,0.0,5.0,2,7.0,7,7.0,6,7,7,0


In [155]:
# Veo cuantas personas a predecir pierdo
cant_modelos_predecir = pd.merge(cant_modelos, df_persons, on="person", how="inner")
cant_modelos_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [156]:
# Separamos la variable a predecir

X, y = df_con_modelos_num.iloc[:,:-1], df_con_modelos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [157]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [158]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [159]:
# Predecimos

preds = xg_reg.predict(X_test)

In [160]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8276350524756938
Test acuracy:  0.8323461241308266
ROC auc score:  0.6412577357013607
Confusion matrix: 
[[3150  542]
 [ 109   82]]


## Cantidad de días distintos de entrada al sitio

In [161]:
df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()
cant_dias_dist.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,person,cant_dias_dist
0,0008ed71,1
1,00091926,22
2,00091a7a,1
3,000ba417,3
4,000c79fe,1


In [162]:
# Merge 

df_con_dias_dist = pd.merge(cant_dias_dist, df_con_modelos, on="person", how="inner")
df_con_dias_dist_num = df_con_dias_dist.drop(columns=["person", "modelos_dist"])
df_con_dias_dist_num.head()

,cant_dias_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,1,0,2.0,0.0,0.0,1.0,3,6.0,6,6.0,5,6,6,0
1,1,0,1.0,1.0,1.0,4.0,9,17.0,17,17.0,16,17,17,0
2,1,0,1.0,5.0,0.0,3.0,5,19.0,19,19.0,18,19,19,0
3,14,114,12.0,17.0,12.0,63.0,189,290.0,471,290.0,358,290,72,0
4,1,0,0.0,0.0,0.0,5.0,2,7.0,7,7.0,6,7,7,0


In [163]:
# Veo cuantas personas a predecir pierdo
cant_dias_dist_predecir = pd.merge(cant_dias_dist, df_persons, on="person", how="inner")
cant_dias_dist_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [164]:
# Separamos la variable a predecir

X, y = df_con_dias_dist_num.iloc[:,:-1], df_con_dias_dist_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [165]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 1, min_child_weight=1)


In [166]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [167]:
# Predecimos

preds = xg_reg.predict(X_test)

In [168]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8376794797501771
Test acuracy:  0.837496780839557
ROC auc score:  0.6290720561792017
Confusion matrix: 
[[3176  516]
 [ 115   76]]


In [176]:
# Separamos la variable a predecir

X, y = df_con_dias_dist_num.iloc[:,:-1],df_con_dias_dist_num.iloc[:,-1]

In [177]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [178]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [195]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 4, n_estimators = 6, scale_pos_weight = 11, min_child_weight=1)


In [196]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=4, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=11, seed=None,
       silent=True, subsample=1)

In [197]:
# Predecimos

preds = xg_reg.predict_proba(X_test)[:, 1]
preds

array([0.41958648, 0.63762015, 0.30858842, ..., 0.5040561 , 0.39075756,
       0.39075756], dtype=float32)

In [198]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, xg_reg.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, xg_reg.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8376794797501771
Test acuracy:  0.8403296420293588
ROC auc score:  0.7308450987844101
Confusion matrix: 
[[3182  510]
 [ 110   81]]


## Promedio de entradas por dia

In [232]:
entradas_tot = df.groupby("person").agg({"person": "count"})
#cant_entradas_tot = entradas_tot[["person", "entradas_tot"]]
entradas_tot.columns = ["cant_entradas"]
entradas_tot.reset_index(inplace=True)

df_dias = df[["person", "timestamp"]]
df_dias["day_of_year"] = df_dias["timestamp"].dt.dayofyear
cant_dias_dist = df_dias.groupby("person")["day_of_year"].nunique().rename("cant_dias_dist").reset_index()

entradas_tot_y_dias = pd.merge(entradas_tot, cant_dias_dist, on="person", how="inner")
entradas_tot_y_dias["promedio_por_dia"] = (entradas_tot_y_dias["cant_dias_dist"] / entradas_tot_y_dias["cant_entradas"])
entradas_tot_y_dias.head()

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,person,cant_entradas,cant_dias_dist,promedio_por_dia
0,0008ed71,6,1,0.166667
1,00091926,448,22,0.049107
2,00091a7a,10,1,0.100000
3,000ba417,206,3,0.014563
4,000c79fe,17,1,0.058824


In [233]:
entradas_tot_y_dias = entradas_tot_y_dias[["person", "promedio_por_dia"]]

In [234]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_dias_predecir = pd.merge(entradas_tot_y_dias, df_persons, on="person", how="inner")
entradas_tot_y_dias_predecir["person"].count() - df_persons["person"].count()

0

## Promedio de entradas por mes

In [235]:
df_mes = df[["person", "month"]]
cant_meses_dist = df_mes.groupby("person")["month"].nunique().rename("cant_meses_dist").reset_index()

entradas_tot_y_meses = pd.merge(entradas_tot, cant_meses_dist, on="person", how="inner")
entradas_tot_y_meses["promedio_por_mes"] = (entradas_tot_y_meses["cant_meses_dist"] / entradas_tot_y_meses["cant_entradas"])
entradas_tot_y_meses = entradas_tot_y_meses[["person", "promedio_por_mes"]]

entradas_tot_y_meses.head()

,person,promedio_por_mes
0,0008ed71,0.166667
1,00091926,0.002232
2,00091a7a,0.100000
3,000ba417,0.004854
4,000c79fe,0.058824


In [236]:
# Merge

labels_con_promedios = pd.merge(entradas_tot_y_meses, entradas_tot_y_dias, on="person", how="inner")
labels_con_promedios = pd.merge(labels_con_promedios, df_con_dias_dist, on="person", how="inner")
labels_con_promedios_num = labels_con_promedios.drop(columns="person")
labels_con_promedios_num.head()

,promedio_por_mes,promedio_por_dia,cant_dias_dist,modelos_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0.166667,0.166667,1,3,0,2.0,0.0,0.0,1.0,3,6.0,6,6.0,5,6,6,0
1,0.058824,0.058824,1,1,0,1.0,1.0,1.0,4.0,9,17.0,17,17.0,16,17,17,0
2,0.052632,0.052632,1,2,0,1.0,5.0,0.0,3.0,5,19.0,19,19.0,18,19,19,0
3,0.008493,0.029724,14,26,114,12.0,17.0,12.0,63.0,189,290.0,471,290.0,358,290,72,0
4,0.142857,0.142857,1,1,0,0.0,0.0,0.0,5.0,2,7.0,7,7.0,6,7,7,0


In [237]:
# Veo cuantas personas a predecir pierdo
entradas_tot_y_meses_predecir = pd.merge(entradas_tot_y_meses, df_persons, on="person", how="inner")
entradas_tot_y_meses_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [238]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [239]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=5)


In [240]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=5, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [241]:
# Predecimos

preds = xg_reg.predict(X_test)

In [242]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8460498358122465
Test acuracy:  0.8411022405356683
ROC auc score:  0.6334504206066037
Confusion matrix: 
[[3189  503]
 [ 114   77]]


In [243]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8460498358122465
Test acuracy:  0.8411022405356683

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.86      0.91      3692
           1       0.13      0.40      0.20       191

   micro avg       0.84      0.84      0.84      3883
   macro avg       0.55      0.63      0.56      3883
weighted avg       0.92      0.84      0.88      3883


Precision Score:  0.13275862068965516
Recall Score:  0.4031413612565445
F1 Score:  0.19974059662775617
Cohen Kappa Score:  0.1357829590802795
ROC auc score:  0.6334504206066037
Confusion matrix: 
[[3189  503]
 [ 114   77]]


In [244]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1],labels_con_promedios_num.iloc[:,-1]

In [245]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [246]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [247]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 6, min_child_weight=1)


In [248]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=1)

In [249]:
# Predecimos

preds = xg_reg.predict_proba(X_test)[:, 1]
preds

array([0.36512432, 0.54599017, 0.28901175, ..., 0.42871228, 0.34284964,
       0.34284964], dtype=float32)

In [250]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, xg_reg.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, xg_reg.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9318781791256198
Test acuracy:  0.9296935359258306
ROC auc score:  0.7383659589433501
Confusion matrix: 
[[3576  116]
 [ 157   34]]


### Duplico los compradores (label = 1)

### Prueba

In [251]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [252]:
# Duplico los compradores

y_train_df = y_train.to_frame()
X_train_df = X_train.reset_index()
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [253]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [254]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [255]:
# Predecimos

preds = xg_reg.predict(X_test)

In [256]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.6244485294117647
Test acuracy:  0.6144733453515323
ROC auc score:  0.6805928199077671
Confusion matrix: 
[[2242 1450]
 [  47  144]]


In [257]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6244485294117647
Test acuracy:  0.6144733453515323

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.61      0.75      3692
           1       0.09      0.75      0.16       191

   micro avg       0.61      0.61      0.61      3883
   macro avg       0.53      0.68      0.46      3883
weighted avg       0.94      0.61      0.72      3883


Precision Score:  0.0903387703889586
Recall Score:  0.7539267015706806
F1 Score:  0.16134453781512606
Cohen Kappa Score:  0.08057198651049235
ROC auc score:  0.6805928199077671
Confusion matrix: 
[[2242 1450]
 [  47  144]]


### Usando SMOTE

In [258]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [259]:
cant_ceros = np.sum(y == 0)
cant_unos = np.sum(y == 1)
cant_ceros, cant_unos

(18434, 980)

In [260]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors=90, sampling_strategy={0: 18434, 1: 2500 })
X_train_res, y_train_res = sm.fit_sample(X_train, y_train.ravel())
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

C:\Users\seba\Anaconda3\lib\site-packages\imblearn\utils\_validation.py:257: UserWarning: After over-sampling, the number of samples (18434) in class 0 will be larger than the number of samples in the majority class (class #0 -> 14742)
  n_samples_majority))


In [261]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [262]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [263]:
# Predecimos

preds = xg_reg.predict(X_test)

In [264]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.5649183147033534
Test acuracy:  0.5570435230491888
ROC auc score:  0.6305334017799912
Confusion matrix: 
[[2027 1665]
 [  55  136]]


In [265]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.5649183147033534
Test acuracy:  0.5570435230491888

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.55      0.70      3692
           1       0.08      0.71      0.14       191

   micro avg       0.56      0.56      0.56      3883
   macro avg       0.52      0.63      0.42      3883
weighted avg       0.93      0.56      0.67      3883


Precision Score:  0.07551360355358135
Recall Score:  0.7120418848167539
F1 Score:  0.13654618473895583
Cohen Kappa Score:  0.052248673682274704
ROC auc score:  0.6305334017799912
Confusion matrix: 
[[2027 1665]
 [  55  136]]


### Random oversampling

In [266]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [267]:
cant_ceros = np.sum(y == 0)
cant_unos = np.sum(y == 1)
cant_ceros, cant_unos

(18434, 980)

In [268]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 18434, 1: 1219})
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [269]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [270]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [271]:
# Predecimos

preds = xg_reg.predict(X_test)

In [272]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.750826845774182
Test acuracy:  0.758434200360546
ROC auc score:  0.7116143295536408
Confusion matrix: 
[[2819  873]
 [  65  126]]


In [273]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.750826845774182
Test acuracy:  0.758434200360546

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.76      0.86      3692
           1       0.13      0.66      0.21       191

   micro avg       0.76      0.76      0.76      3883
   macro avg       0.55      0.71      0.53      3883
weighted avg       0.94      0.76      0.83      3883


Precision Score:  0.12612612612612611
Recall Score:  0.6596858638743456
F1 Score:  0.21176470588235294
Cohen Kappa Score:  0.1408059913869566
ROC auc score:  0.7116143295536408
Confusion matrix: 
[[2819  873]
 [  65  126]]


In [470]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1],labels_con_promedios_num.iloc[:,-1]

In [471]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [472]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [473]:
cant_ceros = np.sum(y == 0)
cant_unos = np.sum(y == 1)
cant_ceros, cant_unos

(18434, 980)

In [474]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 18434, 1: 1219})
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [491]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 4, min_child_weight=1)


In [492]:
#Entrenamos

xg_reg.fit(X_train_res_df,y_train_res)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=4, seed=None,
       silent=True, subsample=1)

In [493]:
# Predecimos

preds = xg_reg.predict_proba(X_test)[:, 1]
preds

array([0.3594419 , 0.5228824 , 0.2852726 , ..., 0.408071  , 0.3316679 ,
       0.31432545], dtype=float32)

In [494]:
# Vemos que onda

train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, xg_reg.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, xg_reg.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9334452755304533
Test acuracy:  0.9464331702292043
ROC auc score:  0.7834819022876689
Confusion matrix: 
[[3643   49]
 [ 159   32]]


### Random undersampling

In [389]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1], labels_con_promedios_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [390]:
cant_ceros = np.sum(y == 0)
cant_unos = np.sum(y == 1)
cant_ceros, cant_unos

(18434, 980)

In [446]:
from imblearn.under_sampling import RandomUnderSampler

ros = RandomUnderSampler(random_state=2, sampling_strategy ={0: 15434, 1: 980 })
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [447]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

In [448]:
#Entrenamos

xg_reg.fit(X_train_res_df ,y_train_res.ravel())

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [449]:
# Predecimos

preds = xg_reg.predict(X_test)

In [450]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8069940294870233
Test acuracy:  0.8127736286376513

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.83      0.89      3692
           1       0.14      0.56      0.23       191

   micro avg       0.81      0.81      0.81      3883
   macro avg       0.56      0.69      0.56      3883
weighted avg       0.93      0.81      0.86      3883


Precision Score:  0.14266666666666666
Recall Score:  0.5602094240837696
F1 Score:  0.22741764080765142
Cohen Kappa Score:  0.16168602332420556
ROC auc score:  0.6930245386941059
Confusion matrix: 
[[3049  643]
 [  84  107]]


## Cantidad de eventos particulares

In [291]:
df_events = df[["person", "event"]]
cant_por_evento = pd.concat([pd.get_dummies(df_events['event']),df_events[['person']]],axis = 1).groupby('person')\
    .sum().reset_index()
cant_por_evento_short = cant_por_evento[["person", "ad campaign hit", "search engine hit"]]

# Merge ... "search engine hit", "generic listing", "searched products" "ad campaign hit", "brand listing"

labels_con_eventos = pd.merge(cant_por_evento_short, labels_con_promedios, on="person", how="inner")
labels_con_eventos_num = labels_con_eventos.drop(columns="person")
labels_con_eventos_num.head()

,ad campaign hit,search engine hit,promedio_por_mes,promedio_por_dia,cant_dias_dist,modelos_dist,distan_dias,cant_city,cant_src,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0.0,0.0,0.166667,0.166667,1,3,0,2.0,0.0,0.0,1.0,3,6.0,6,6.0,5,6,6,0
1,1.0,1.0,0.058824,0.058824,1,1,0,1.0,1.0,1.0,4.0,9,17.0,17,17.0,16,17,17,0
2,5.0,0.0,0.052632,0.052632,1,2,0,1.0,5.0,0.0,3.0,5,19.0,19,19.0,18,19,19,0
3,29.0,13.0,0.008493,0.029724,14,26,114,12.0,17.0,12.0,63.0,189,290.0,471,290.0,358,290,72,0
4,0.0,0.0,0.142857,0.142857,1,1,0,0.0,0.0,0.0,5.0,2,7.0,7,7.0,6,7,7,0


In [292]:
# Veo cuantas personas a predecir pierdo
cant_por_evento_short_predecir = pd.merge(cant_por_evento_short, df_persons, on="person", how="inner")
cant_por_evento_short_predecir["person"].count() - df_persons["person"].count()

0

### Prueba

In [293]:
# Separamos la variable a predecir

X, y = labels_con_eventos_num.iloc[:,:-1], labels_con_eventos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [294]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=15)


In [295]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=15, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [296]:
# Predecimos

preds = xg_reg.predict(X_test)

In [297]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8313695190264632
Test acuracy:  0.8328611898016998
ROC auc score:  0.6365638454164374
Confusion matrix: 
[[3154  538]
 [ 111   80]]


In [309]:
# Separamos la variable a predecir

X, y = labels_con_eventos_num.iloc[:,:-1],labels_con_eventos_num.iloc[:,-1]

In [310]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [311]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [312]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=1)


In [313]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [314]:
# Predecimos

preds = xg_reg.predict_proba(X_test)[:, 1]
preds

array([0.39336428, 0.61340684, 0.29059574, ..., 0.44678748, 0.35029072,
       0.3423286 ], dtype=float32)

In [315]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, xg_reg.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, xg_reg.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9086343442147962
Test acuracy:  0.9090909090909091
ROC auc score:  0.7412716330200292
Confusion matrix: 
[[3483  209]
 [ 144   47]]


## Visitas a productos segun condicion, color y almacenamiento de producto.¶
copiado de santi

In [316]:
df_events_visitas_prod = df.loc[df['event'] == 'viewed product']
df_events_visitas_prod = df_events_visitas_prod[['person','storage','color','condition']]

df_events_visitas_prod_condition_storage = pd.concat([df_events_visitas_prod['person'],\
           pd.get_dummies(df_events_visitas_prod['storage']),\
           pd.get_dummies(df_events_visitas_prod['condition'])],axis = 1).groupby('person').sum().reset_index()

In [317]:
top_10_colores = df_events_visitas_prod['color'].value_counts().head(10).index


df_events_visitas_prod_color = df_events_visitas_prod.loc[df_events_visitas_prod['color'].isin(top_10_colores)]
df_events_visitas_prod_color = pd.concat([df_events_visitas_prod_color['person'],\
                                          pd.get_dummies(df_events_visitas_prod_color['color'])],axis = 1)

df_events_visitas_prod_color = df_events_visitas_prod_color.groupby('person').sum().reset_index()

df_events_visitas_prod_color_storage_condition = pd.merge(df_events_visitas_prod_condition_storage,\
                        df_events_visitas_prod_color, on = 'person', how = 'inner')


df_events_visitas_prod_color_storage_condition = pd.merge(df_events_visitas_prod_color_storage_condition, todas_las_personas, on="person", how="right").fillna(0)
df_events_visitas_prod_color_storage_condition.head()

,person,128GB,16GB,256GB,32GB,4GB,512MB,64GB,8GB,Bom,...,Azul,Branco,Cinza espacial,Dourado,Ouro Rosa,Prata,Prateado,Preto,Preto Matte,Rosa
0,00091926,48.0,104.0,10.0,132.0,0.0,0.0,78.0,0.0,102.0,...,3.0,3.0,65.0,101.0,28.0,1.0,43.0,79.0,29.0,3.0
1,00091a7a,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,000ba417,0.0,108.0,1.0,20.0,1.0,0.0,1.0,22.0,110.0,...,4.0,15.0,1.0,14.0,0.0,0.0,0.0,71.0,0.0,7.0
3,000c79fe,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
4,000e4d9e,1.0,108.0,1.0,208.0,0.0,0.0,21.0,0.0,124.0,...,23.0,85.0,14.0,46.0,2.0,6.0,1.0,149.0,0.0,9.0


In [318]:
# Veo cuantas personas a predecir pierdo
visitas_prod_color_storage_condition_predecir = pd.merge(df_events_visitas_prod_color_storage_condition, df_persons, on="person", how="inner")
visitas_prod_color_storage_condition_predecir["person"].count() - df_persons["person"].count()

0

In [319]:
# Merge

labels_con_storage_color = pd.merge(df_events_visitas_prod_color_storage_condition, labels_con_eventos, on="person", how="inner")
labels_con_storage_color_num = labels_con_storage_color.drop(columns="person")
labels_con_storage_color_num.head()

,128GB,16GB,256GB,32GB,4GB,512MB,64GB,8GB,Bom,Bom - Sem Touch ID,...,cant_eng,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label
0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,1.0,4.0,9,17.0,17,17.0,16,17,17,0
1,0.0,0.0,2.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,...,0.0,3.0,5,19.0,19,19.0,18,19,19,0
2,0.0,94.0,0.0,14.0,0.0,0.0,0.0,81.0,105.0,0.0,...,12.0,63.0,189,290.0,471,290.0,358,290,72,0
3,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,5.0,2,7.0,7,7.0,6,7,7,0
4,8.0,19.0,4.0,11.0,0.0,0.0,10.0,0.0,9.0,0.0,...,6.0,20.0,52,54.0,96,54.0,64,54,17,0


### Prueba

In [320]:
# Separamos la variable a predecir

X, y = labels_con_storage_color_num.iloc[:,:-1], labels_con_storage_color_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [321]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=15)


In [322]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=15, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [323]:
# Predecimos

preds = xg_reg.predict(X_test)

In [324]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8342025626167021
Test acuracy:  0.8269379345866598
ROC auc score:  0.6305553149094689
Confusion matrix: 
[[3131  558]
 [ 114   80]]


# Entro como 'new' en el mes 5

In [325]:
df_mes_5 = df[["person", "month", "new_vs_returning"]].loc[df["month"] == 5]
df_mes_5["is_new"] = (df_mes_5["new_vs_returning"] == "New") * 1
df_mew_5 = df_mes_5[["person", "is_new"]]
df_mew_5 = df_mew_5.sort_values("is_new", ascending=True)
df_mew_5 = df_mew_5.drop_duplicates(subset=['person'])
df_mew_5 = pd.merge(df_mew_5, todas_las_personas, on="person", how="right").fillna(0)
df_mew_5.head()

,person,is_new
0,4886f805,0.0
1,d6288188,0.0
2,4858040e,0.0
3,a6538d3e,0.0
4,b884c881,0.0


In [326]:
# Veo cuantas personas a predecir pierdo
df_mew_5_predecir = pd.merge(df_mew_5, df_persons, on="person", how="inner")
df_mew_5_predecir["person"].count() - df_persons["person"].count()

0

In [327]:
# Merge

labels_con_new_mes5 = pd.merge(df_mew_5, labels_con_storage_color, on="person", how="inner")
labels_con_new_mes5_num = labels_con_new_mes5.drop(columns="person")

### Prueba

In [328]:
# Separamos la variable a predecir

X, y = labels_con_new_mes5_num.iloc[:,:-1], labels_con_new_mes5_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [329]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=4)


In [330]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=4, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [331]:
# Predecimos

preds = xg_reg.predict(X_test)

In [332]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8540982551026979
Test acuracy:  0.8524336852948751
ROC auc score:  0.6102463712757831
Confusion matrix: 
[[3246  450]
 [ 123   64]]


# -- Agrego cosas de Santi --

## Cantidad de visitas a los 15 modelos mas visitados

In [333]:
df_visitas_producto = df.loc[df['event'] == 'viewed product']
top_15 = df_visitas_producto['model'].value_counts().head(15).index
df_15_productos_mas_visitados = df_visitas_producto.loc[df_visitas_producto['model'].isin(top_15)]

df_15_productos_mas_visitados = df_15_productos_mas_visitados[['person','model']]
df_15_productos_mas_visitados.head()

,person,model
1,ad93850f,iPhone 5s
2,0297fc1e,iPhone 6S
3,2d681dd8,iPhone 7
6,1b9f7cf6,iPhone 6
7,29ebb414,iPhone 6 Plus


In [334]:
df_15_productos_mas_visitados = pd.concat([df_15_productos_mas_visitados,\
                    pd.get_dummies(df_15_productos_mas_visitados['model'])],axis = 1)

df_15_productos_mas_visitados = df_15_productos_mas_visitados.groupby('person').sum().reset_index()
df_15_productos_mas_visitados.head()

,person,Samsung Galaxy J5,Samsung Galaxy S6 Edge,Samsung Galaxy S6 Flat,Samsung Galaxy S7,Samsung Galaxy S7 Edge,Samsung Galaxy S8,iPhone 5c,iPhone 5s,iPhone 6,iPhone 6 Plus,iPhone 6S,iPhone 6S Plus,iPhone 7,iPhone 7 Plus,iPhone SE
0,00091926,1.0,3.0,15.0,1.0,9.0,5.0,0.0,0.0,5.0,41.0,94.0,51.0,45.0,9.0,3.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,000ba417,11.0,0.0,1.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,000e4d9e,1.0,5.0,139.0,22.0,2.0,9.0,0.0,7.0,0.0,0.0,1.0,0.0,1.0,0.0,11.0


In [335]:
# vis = visitas
nombres_columnas = ['vis_'+columna for columna in df_15_productos_mas_visitados.columns]
nombres_columnas[0] = 'person'
df_15_productos_mas_visitados.columns = nombres_columnas

df_15_productos_mas_visitados = pd.merge(df_15_productos_mas_visitados, todas_las_personas, on="person", how="right").fillna(0)

df_15_productos_mas_visitados.head()

,person,vis_Samsung Galaxy J5,vis_Samsung Galaxy S6 Edge,vis_Samsung Galaxy S6 Flat,vis_Samsung Galaxy S7,vis_Samsung Galaxy S7 Edge,vis_Samsung Galaxy S8,vis_iPhone 5c,vis_iPhone 5s,vis_iPhone 6,vis_iPhone 6 Plus,vis_iPhone 6S,vis_iPhone 6S Plus,vis_iPhone 7,vis_iPhone 7 Plus,vis_iPhone SE
0,00091926,1.0,3.0,15.0,1.0,9.0,5.0,0.0,0.0,5.0,41.0,94.0,51.0,45.0,9.0,3.0
1,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0
2,000ba417,11.0,0.0,1.0,0.0,0.0,0.0,6.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0
4,000e4d9e,1.0,5.0,139.0,22.0,2.0,9.0,0.0,7.0,0.0,0.0,1.0,0.0,1.0,0.0,11.0


In [336]:
# Veo cuantas personas a predecir pierdo
df_15_productos_mas_visitados_predecir = pd.merge(df_15_productos_mas_visitados, df_persons, on="person", how="inner")
df_15_productos_mas_visitados_predecir["person"].count() - df_persons["person"].count()

0

In [337]:
df_train_sin_labels = pd.merge(df_15_productos_mas_visitados,cant_por_evento,on = 'person',how = 'outer')

df_train_sin_labels.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38829 entries, 0 to 38828
Data columns (total 27 columns):
person                        38829 non-null object
vis_Samsung Galaxy J5         38829 non-null float64
vis_Samsung Galaxy S6 Edge    38829 non-null float64
vis_Samsung Galaxy S6 Flat    38829 non-null float64
vis_Samsung Galaxy S7         38829 non-null float64
vis_Samsung Galaxy S7 Edge    38829 non-null float64
vis_Samsung Galaxy S8         38829 non-null float64
vis_iPhone 5c                 38829 non-null float64
vis_iPhone 5s                 38829 non-null float64
vis_iPhone 6                  38829 non-null float64
vis_iPhone 6 Plus             38829 non-null float64
vis_iPhone 6S                 38829 non-null float64
vis_iPhone 6S Plus            38829 non-null float64
vis_iPhone 7                  38829 non-null float64
vis_iPhone 7 Plus             38829 non-null float64
vis_iPhone SE                 38829 non-null float64
ad campaign hit               38829 non-null

In [338]:
# probar 2 alternativas
df_train_sin_labels1 = df_train_sin_labels.dropna()
df_train_sin_labels2 = df_train_sin_labels.fillna(0)

In [339]:
# Merge

labels_con_15_modelos = pd.merge(df_15_productos_mas_visitados, labels_con_new_mes5, on="person", how="inner")
labels_con_15_modelos_num = labels_con_15_modelos.drop(columns="person")

### Prueba

In [340]:
# Separamos la variable a predecir

X, y = labels_con_15_modelos_num.iloc[:,:-1], labels_con_15_modelos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [341]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=10)


In [342]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [343]:
# Predecimos

preds = xg_reg.predict(X_test)

In [344]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8465649346468354
Test acuracy:  0.8403296420293588
ROC auc score:  0.6333179732429994
Confusion matrix: 
[[3184  503]
 [ 117   79]]


--- Aca bajó de 65 a 60!! ---

## Estado(region) del usuario

In [345]:
top_20_estados = list(df['region'].value_counts().head(21).index)
del top_20_estados[1]
top_20_estados

['Sao Paulo',
 'Minas Gerais',
 'Rio de Janeiro',
 'Bahia',
 'Pernambuco',
 'Ceara',
 'Parana',
 'Rio Grande do Sul',
 'Espirito Santo',
 'Federal District',
 'Maranhao',
 'Goias',
 'Santa Catarina',
 'Para',
 'Rio Grande do Norte',
 'Paraíba',
 'Piaui',
 'Alagoas',
 'Sergipe',
 'Amazonas']

In [346]:
df_estado = df[['person','region','month']]
df_estado = df_estado.sort_values('month',ascending = True).dropna()

df_estado = df_estado.loc[df_estado['region'].isin(top_20_estados)]


df_estado = df_estado.drop_duplicates(subset = 'person', keep = 'last').drop(columns = ['month'])

df_estado.head()

,person,region
2151588,0869b313,Pernambuco
2150498,78561ee9,Sao Paulo
2294065,540e67b1,Pernambuco
2248029,041627fe,Federal District
2151706,62acae45,Minas Gerais


In [347]:
# PROBAR 2 OPCIONES (LabelEncoder o con One hot encoding)
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
df_estado1 = df_estado.copy()

In [348]:
df_estado1['region'] = le.fit_transform(df_estado1['region'])
df_estado1 = pd.merge(df_estado1, todas_las_personas, on="person", how="right").fillna(0)

df_estado1.head()

,person,region
0,0869b313,12.0
1,78561ee9,18.0
2,540e67b1,12.0
3,041627fe,5.0
4,62acae45,8.0


In [349]:
df_estado2 = 0
df_estado2 = pd.concat([df_estado['person'],pd.get_dummies(df_estado['region'])],axis=1)
df_estado2 = pd.merge(df_estado2, todas_las_personas, on="person", how="right").fillna(0)
df_estado2.head()

,person,Alagoas,Amazonas,Bahia,Ceara,Espirito Santo,Federal District,Goias,Maranhao,Minas Gerais,...,Parana,Paraíba,Pernambuco,Piaui,Rio Grande do Norte,Rio Grande do Sul,Rio de Janeiro,Santa Catarina,Sao Paulo,Sergipe
0,0869b313,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,78561ee9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,540e67b1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,041627fe,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,62acae45,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [350]:
# Veo cuantas personas a predecir pierdo
df_estado2_predecir = pd.merge(df_estado2, df_persons, on="person", how="inner")
df_estado2_predecir["person"].count() - df_persons["person"].count()

0

In [351]:
# Veo cuantas personas a predecir pierdo
df_estado1_predecir = pd.merge(df_estado1, df_persons, on="person", how="inner")
df_estado1_predecir["person"].count() - df_persons["person"].count()

0

In [352]:
# Merge

labels_con_regiones = pd.merge(df_estado2, labels_con_15_modelos, on="person", how="inner")
labels_con_regiones_num = labels_con_regiones.drop(columns="person")

### Prueba

In [353]:
# Separamos la variable a predecir

X, y = labels_con_regiones_num.iloc[:,:-1], labels_con_regiones_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [354]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [355]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [356]:
# Predecimos

preds = xg_reg.predict(X_test)

In [357]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8478526817333076
Test acuracy:  0.8405871748647953
ROC auc score:  0.6250383146701501
Confusion matrix: 
[[3191  503]
 [ 116   73]]


 --- ¿¿¿?? aca se cagó todo, paso a 0.5, wtf. Tanto con LE como con OHE ---

## Cantidad de veces que accede desde cada dispositivo

In [358]:
df_device = df.loc[df['event'] == 'visited site']

df_device = df_device[['person','device_type']]

df_device = pd.concat([df_device['person'],pd.get_dummies(df_device['device_type'])],axis = 1)

df_device.drop(columns = ['Unknown'],inplace = True)

df_device = df_device.groupby('person').sum().reset_index()

df_device = pd.merge(df_device, todas_las_personas, on="person", how="right").fillna(0)


df_device.head()

,person,Computer,Smartphone,Tablet
0,0008ed71,2.0,0.0,0.0
1,00091926,34.0,0.0,0.0
2,00091a7a,0.0,1.0,0.0
3,000ba417,6.0,0.0,0.0
4,000c79fe,0.0,1.0,0.0


In [359]:
# Lo mismo se podria hacer pero considerando las visitas a producto desde cada movil.
# Ver.

In [360]:
# Veo cuantas personas a predecir pierdo
df_device_predecir = pd.merge(df_device, df_persons, on="person", how="inner")
df_device_predecir["person"].count() - df_persons["person"].count()

0

In [361]:
# Merge

labels_con_dispositivos = pd.merge(df_device, labels_con_15_modelos, on="person", how="inner")
labels_con_dispositivos_num = labels_con_dispositivos.drop(columns="person")

### Prueba

In [362]:
# Separamos la variable a predecir

X, y = labels_con_dispositivos_num.iloc[:,:-1], labels_con_dispositivos_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [363]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=20)


In [364]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=20, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [365]:
# Predecimos

preds = xg_reg.predict(X_test)

In [366]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8575751722361729
Test acuracy:  0.8465104300798352
ROC auc score:  0.5995195305629726
Confusion matrix: 
[[3221  458]
 [ 138   66]]


## Visitas a la pagina segun resolucion

In [367]:
df_resolucion = df.loc[df['event'] == 'visited site']
df_resolucion = df_resolucion[['person','screen_resolution']].dropna()

top_5_resoluciones = df_resolucion['screen_resolution'].value_counts().head(10).index
df_resolucion = df_resolucion.loc[df_resolucion['screen_resolution'].isin(top_5_resoluciones)]
df_resolucion = pd.concat([df_resolucion['person'],\
        pd.get_dummies(df_resolucion['screen_resolution'])],axis = 1).groupby('person').sum().reset_index()

df_resolucion = pd.merge(df_resolucion, todas_las_personas, on="person", how="right").fillna(0)

df_resolucion.head()

,person,1024x768,1360x768,1366x768,1440x900,1600x900,1920x1080,320x534,320x568,320x570,360x640
0,0008ed71,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
1,00091926,34.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000ba417,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [368]:
df_resolucion.describe()

,1024x768,1360x768,1366x768,1440x900,1600x900,1920x1080,320x534,320x568,320x570,360x640
count,38829.000000,38829.000000,38829.000000,38829.000000,38829.000000,38829.000000,38829.000000,38829.000000,38829.000000,38829.000000
mean,0.154266,0.170285,1.281516,0.142368,0.158129,0.204692,0.196554,0.114116,0.133199,1.886065
std,1.700296,1.907744,5.156652,1.606356,1.929644,2.114615,1.843201,1.318921,1.453222,6.464359
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,105.000000,133.000000,246.000000,99.000000,116.000000,146.000000,124.000000,78.000000,75.000000,293.000000


In [369]:
# Creo que es mucho 20 resoluciones, probar (quizas empeore)..

In [370]:
# Veo cuantas personas a predecir pierdo
df_resolucion_predecir = pd.merge(df_resolucion, df_persons, on="person", how="inner")
df_resolucion_predecir["person"].count() - df_persons["person"].count()

0

In [371]:
# Merge

labels_con_resoluciones = pd.merge(df_resolucion, labels_con_dispositivos, on="person", how="inner")
labels_con_resoluciones_num = labels_con_resoluciones.drop(columns="person")

In [372]:
labels_con_resoluciones.shape

(19414, 72)

In [373]:
# Super Merge
"""dfs=0
dfs =[mes_dia_mas_entradas, entradas_30_dias_df, df_con_mes_5, entradas_tot,\
      cant_checkouts_tot, mayor_ciudad, mayor_camp, mayor_engine, mayor_modelo, mayor_evento, df_con_check\
     , distancia_dias, cant_modelos, cant_dias_dist, entradas_tot_y_dias, entradas_tot_y_meses,\
      df_events_visitas_prod_color_storage_condition, df_mew_5, df_device, df_resolucion,\
      df_sistema]

df_mejor = pd.concat(dfs, axis=1).reset_index()"""


'dfs=0\ndfs =[mes_dia_mas_entradas, entradas_30_dias_df, df_con_mes_5, entradas_tot,      cant_checkouts_tot, mayor_ciudad, mayor_camp, mayor_engine, mayor_modelo, mayor_evento, df_con_check     , distancia_dias, cant_modelos, cant_dias_dist, entradas_tot_y_dias, entradas_tot_y_meses,      df_events_visitas_prod_color_storage_condition, df_mew_5, df_device, df_resolucion,      df_sistema]\n\ndf_mejor = pd.concat(dfs, axis=1).reset_index()'

### Prueba

In [374]:
# Separamos la variable a predecir

X, y = labels_con_resoluciones_num.iloc[:,:-1], labels_con_resoluciones_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [375]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=16)


In [376]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=16, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [377]:
# Predecimos

preds = xg_reg.predict(X_test)

In [378]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.850106239134634
Test acuracy:  0.84393510172547
ROC auc score:  0.6774540894209073
Confusion matrix: 
[[3173  498]
 [ 108  104]]


In [386]:
# Separamos la variable a predecir

X, y = labels_con_resoluciones_num.iloc[:,:-1],labels_con_resoluciones_num.iloc[:,-1]

In [387]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [388]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [397]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 7, min_child_weight=1)


In [398]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=7, seed=None,
       silent=True, subsample=1)

In [399]:
# Predecimos

preds = xg_reg.predict_proba(X_test)[:, 1]
preds

array([0.33288836, 0.28972083, 0.4415593 , ..., 0.31668237, 0.4630233 ,
       0.3611661 ], dtype=float32)

In [400]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, xg_reg.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, xg_reg.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9370935548258322
Test acuracy:  0.9255730105588462
ROC auc score:  0.7726057883564708
Confusion matrix: 
[[3554  117]
 [ 172   40]]


 --- Aumento un toke, bien ---

## Visitas a la pagina segun sistema operativo

In [495]:
df_sistema = df.loc[df['event'] == 'visited site']
df_sistema = df_sistema[['person','operating_system_version']].dropna()

top_5_sistemas = df_sistema['operating_system_version'].value_counts().head(5).index
df_sistema = df_sistema.loc[df_sistema['operating_system_version'].isin(top_5_sistemas)]
df_sistema = pd.concat([df_sistema['person'],\
        pd.get_dummies(df_sistema['operating_system_version'])],axis = 1).groupby('person').sum().reset_index()

df_sistema = pd.merge(df_sistema, todas_las_personas, on="person", how="right").fillna(0)
df_sistema.head()

,person,Android 6,Android 6.0.1,Android 7,Windows 10,Windows 7
0,0008ed71,0.0,0.0,0.0,2.0,0.0
1,00091926,0.0,0.0,0.0,0.0,34.0
2,000ba417,0.0,0.0,0.0,6.0,0.0
3,000c79fe,0.0,0.0,1.0,0.0,0.0
4,000e4d9e,0.0,0.0,0.0,13.0,0.0


In [496]:
# Veo cuantas personas a predecir pierdo
df_sistema_predecir = pd.merge(df_sistema, df_persons, on="person", how="inner")
df_sistema_predecir["person"].count() - df_persons["person"].count()

0

In [497]:
# Merge

labels_con_SO = pd.merge(df_sistema, labels_con_resoluciones, on="person", how="inner")
labels_con_SO_num = labels_con_SO.drop(columns="person")

In [498]:
labels_con_SO.shape

(19414, 77)

### Prueba

In [499]:
# Separamos la variable a predecir

X, y = labels_con_SO_num.iloc[:,:-1], labels_con_SO_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [500]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=9, scale_pos_weight=ratio)
ratio

18.937098844672658

In [501]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=9, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1,
       scale_pos_weight=18.937098844672658, seed=None, silent=True,
       subsample=1)

In [502]:
# Predecimos

preds = xg_reg.predict(X_test)

In [503]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8540338677483742
Test acuracy:  0.8465104300798352
ROC auc score:  0.6321502211917057
Confusion matrix: 
[[3208  474]
 [ 122   79]]


### Prueba duplicando los compradores

In [504]:
# Separamos la variable a predecir

X, y = labels_con_SO_num.iloc[:,:-1], labels_con_SO_num.iloc[:,-1]

# Convertimos los datos a DMatrix

#data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [505]:
# Duplico los compradores

y_train_df = y_train.to_frame()
X_train_df = X_train.reset_index()
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [506]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=10)
ratio

9.468549422336329

In [507]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=10, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [508]:
# Predecimos

preds = xg_reg.predict(X_test)

In [509]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.703862660944206
Test acuracy:  0.69379345866598
ROC auc score:  0.6503975235176642
Confusion matrix: 
[[2573 1109]
 [  80  121]]


## Visitas a la pagina segun version del navegador

In [510]:
df_navegador = df.loc[df['event'] == 'visited site']
df_navegador = df_navegador[['person','browser_version']].dropna()

top_5_navegadores = df_navegador['browser_version'].value_counts().head(5).index
df_navegador = df_navegador.loc[df_navegador['browser_version'].isin(top_5_navegadores)]
df_navegador = pd.concat([df_navegador['person'],\
        pd.get_dummies(df_navegador['browser_version'])],axis = 1).groupby('person').sum().reset_index()

df_navegador = pd.merge(df_navegador, todas_las_personas, on="person", how="right").fillna(0)

df_navegador.head()

,person,Chrome 65.0,Chrome 66.0,Chrome Mobile 64.0,Chrome Mobile 65.0,Chrome Mobile 66.0
0,0008ed71,0.0,2.0,0.0,0.0,0.0
1,00091926,0.0,34.0,0.0,0.0,0.0
2,000ba417,0.0,6.0,0.0,0.0,0.0
3,000c79fe,0.0,0.0,0.0,0.0,1.0
4,000e619d,0.0,5.0,0.0,0.0,0.0


In [511]:
# Veo cuantas personas a predecir pierdo
df_navegador_predecir = pd.merge(df_navegador, df_persons, on="person", how="inner")
df_navegador_predecir["person"].count() - df_persons["person"].count()

0

In [512]:
# Merge

labels_con_navegador = pd.merge(df_navegador, labels_con_SO, on="person", how="inner")
labels_con_navegador_num = labels_con_navegador.drop(columns="person")

### Prueba

In [513]:
# Separamos la variable a predecir

X, y = labels_con_navegador_num.iloc[:,:-1], labels_con_navegador_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [514]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [515]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [516]:
# Predecimos

preds = xg_reg.predict(X_test)

In [517]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.7969222844633314
Test acuracy:  0.8029873808910636
ROC auc score:  0.6204591087904234
Confusion matrix: 
[[3035  649]
 [ 116   83]]


## Visitas a producto por mes

In [518]:
df_visitas_producto_por_mes = df.loc[df_events['event'] == 'viewed product']
df_visitas_producto_por_mes = df_visitas_producto_por_mes[['person','month']]

df_visitas_producto_por_mes = pd.concat([pd.get_dummies(df_visitas_producto_por_mes['month'])\
                                         ,df_visitas_producto_por_mes[['person']]],axis = 1).groupby('person')\
                                        .sum().reset_index()

df_visitas_producto_por_mes = pd.merge(df_visitas_producto_por_mes, todas_las_personas, on="person", how="right").fillna(0)

df_visitas_producto_por_mes.head()


,person,1,2,3,4,5
0,00091926,0.0,0.0,0.0,0.0,372.0
1,00091a7a,0.0,0.0,3.0,0.0,0.0
2,000ba417,0.0,0.0,0.0,0.0,153.0
3,000c79fe,0.0,0.0,0.0,0.0,3.0
4,000e4d9e,0.0,0.0,0.0,0.0,339.0


In [519]:
nombres_columnas = ['mes_'+str(columna) for columna in df_visitas_producto_por_mes.columns]
nombres_columnas[0] = 'person'
df_visitas_producto_por_mes.columns = nombres_columnas

# Merge 
labels_con_meses = pd.merge(df_visitas_producto_por_mes, labels_con_SO, on="person", how="inner")
labels_con_meses_num = labels_con_meses.drop(columns="person")

In [520]:
# Veo cuantas personas a predecir pierdo
df_visitas_producto_por_mes_predecir = pd.merge(df_visitas_producto_por_mes, df_persons, on="person", how="inner")
df_visitas_producto_por_mes_predecir["person"].count() - df_persons["person"].count()

0

In [521]:
labels_con_meses.columns

Index(['person', 'mes_1', 'mes_2', 'mes_3', 'mes_4', 'mes_5', 'Android 6',
       'Android 6.0.1', 'Android 7', 'Windows 10 ', 'Windows 7 ', '1024x768',
       '1360x768', '1366x768', '1440x900', '1600x900', '1920x1080', '320x534',
       '320x568', '320x570', '360x640', 'Computer', 'Smartphone', 'Tablet',
       'vis_Samsung Galaxy J5', 'vis_Samsung Galaxy S6 Edge',
       'vis_Samsung Galaxy S6 Flat', 'vis_Samsung Galaxy S7',
       'vis_Samsung Galaxy S7 Edge', 'vis_Samsung Galaxy S8', 'vis_iPhone 5c',
       'vis_iPhone 5s', 'vis_iPhone 6', 'vis_iPhone 6 Plus', 'vis_iPhone 6S',
       'vis_iPhone 6S Plus', 'vis_iPhone 7', 'vis_iPhone 7 Plus',
       'vis_iPhone SE', 'is_new', '128GB', '16GB', '256GB', '32GB', '4GB',
       '512MB', '64GB', '8GB', 'Bom', 'Bom - Sem Touch ID', 'Excelente',
       'Muito Bom', 'Novo', 'Azul', 'Branco', 'Cinza espacial', 'Dourado',
       'Ouro Rosa', 'Prata', 'Prateado', 'Preto', 'Preto Matte', 'Rosa',
       'ad campaign hit', 'search engine hit', 'p

### Prueba

In [522]:
# Separamos la variable a predecir

X, y = labels_con_meses_num.iloc[:,:-1], labels_con_meses_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [523]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [524]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [525]:
# Predecimos

preds = xg_reg.predict(X_test)

In [526]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8036185693129869
Test acuracy:  0.8050476435745557
ROC auc score:  0.5891212054880405
Confusion matrix: 
[[3056  626]
 [ 131   70]]


## Dia de la semana que accede

In [527]:
df['day_of_week'] = df["timestamp"].dt.dayofweek
df_dia_visita = df.loc[df['event'] == 'visited site']
df_dia_visita = df_dia_visita[['person','day_of_week']]

df_dia_visita = pd.concat([df_dia_visita['person'],\
                                 pd.get_dummies(df_dia_visita['day_of_week'])],axis = 1)\
                                .groupby('person').sum().reset_index()

df_dia_visita = pd.merge(df_dia_visita, todas_las_personas, on="person", how="right").fillna(0)


df_dia_visita.head()

,person,0,1,2,3,4,5,6
0,0008ed71,0.0,0.0,0.0,2.0,0.0,0.0,0.0
1,00091926,2.0,4.0,3.0,6.0,3.0,5.0,11.0
2,00091a7a,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,4.0,0.0,2.0,0.0
4,000c79fe,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [528]:
# Veo cuantas personas a predecir pierdo
df_dia_visita_predecir = pd.merge(df_dia_visita, df_persons, on="person", how="inner")
df_dia_visita_predecir["person"].count() - df_persons["person"].count()

0

In [529]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2341681 entries, 0 to 2341680
Data columns (total 26 columns):
timestamp                   datetime64[ns]
event                       object
person                      object
url                         object
sku                         float64
model                       object
condition                   object
storage                     object
color                       object
skus                        object
search_term                 object
staticpage                  object
campaign_source             object
search_engine               object
channel                     object
new_vs_returning            object
city                        object
region                      object
country                     object
device_type                 object
screen_resolution           object
operating_system_version    object
browser_version             object
month                       int64
day                         int64
day_o

In [530]:
# Merge

labels_con_dia_semana = pd.merge(df_dia_visita, labels_con_navegador, on="person", how="inner")
labels_con_dia_semana_num = labels_con_dia_semana.drop(columns="person")

### Prueba

In [531]:
# Separamos la variable a predecir

X, y = labels_con_dia_semana_num.iloc[:,:-1], labels_con_dia_semana_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [532]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [533]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [534]:
# Predecimos

preds = xg_reg.predict(X_test)

In [535]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.798467580967098
Test acuracy:  0.787792943600309
ROC auc score:  0.5870534405662237
Confusion matrix: 
[[2985  694]
 [ 130   74]]


In [536]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.798467580967098
Test acuracy:  0.787792943600309

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.81      0.88      3679
           1       0.10      0.36      0.15       204

   micro avg       0.79      0.79      0.79      3883
   macro avg       0.53      0.59      0.52      3883
weighted avg       0.91      0.79      0.84      3883


Precision Score:  0.09635416666666667
Recall Score:  0.3627450980392157
F1 Score:  0.1522633744855967
Cohen Kappa Score:  0.07551145182858243
ROC auc score:  0.5870534405662237
Confusion matrix: 
[[2985  694]
 [ 130   74]]


### Prueba duplicando los compradores

In [537]:
# Separamos la variable a predecir

X, y = labels_con_dia_semana_num.iloc[:,:-1], labels_con_dia_semana_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [538]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [539]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [540]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [541]:
# Predecimos

preds = xg_reg.predict(X_test)

In [542]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6874961672901208
Test acuracy:  0.6901879989698687

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.69      0.81      3679
           1       0.10      0.62      0.17       204

   micro avg       0.69      0.69      0.69      3883
   macro avg       0.54      0.66      0.49      3883
weighted avg       0.93      0.69      0.78      3883


Precision Score:  0.10135674381484437
Recall Score:  0.6225490196078431
F1 Score:  0.17433081674673986
Cohen Kappa Score:  0.09231046651511454
ROC auc score:  0.6582437949357509
Confusion matrix: 
[[2553 1126]
 [  77  127]]


# PREDICCIÓN...

In [543]:
dfs=0
df_final=0
dfs =[mes_dia_mas_entradas, entradas_30_dias_df, entradas_tot,\
      cant_checkouts_tot, mayor_ciudad, mayor_camp, mayor_engine, mayor_modelo, mayor_evento\
     , distancia_dias, cant_modelos, cant_dias_dist, entradas_tot_y_dias, entradas_tot_y_meses,\
      df_events_visitas_prod_color_storage_condition, df_mew_5, df_device, df_resolucion,\
      df_sistema, df_navegador, df_visitas_producto_por_mes, df_dia_visita]

df_final = reduce(lambda left,right: pd.merge(left,right,on='person', how='inner'), dfs)

In [544]:
df_final_prueba =pd.merge(df_final, labels, on="person", how="inner").drop(columns="person")

In [545]:
# Separamos la variable a predecir

X, y = df_final_prueba.iloc[:,:-1], df_final_prueba.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [546]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [547]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [548]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [549]:
# Predecimos

preds = xg_reg.predict(X_test)
preds

array([1., 0., 1., ..., 0., 0., 0.], dtype=float32)

In [550]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.6257055214723927
Test acuracy:  0.6229719289209374
ROC auc score:  0.7023794773306901
Confusion matrix: 
[[2252 1420]
 [  44  167]]


In [551]:
# Random oversampling

In [552]:
# Separamos la variable a predecir

X, y = df_final_prueba.iloc[:,:-1], df_final_prueba.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [553]:
cant_ceros = np.sum(y == 0)
cant_unos = np.sum(y == 1)
cant_ceros, cant_unos

(18434, 980)

In [554]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 18434, 1: 1399})
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [555]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [556]:
#Entrenamos

xg_reg.fit(X_train_res_df,y_train_res)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [557]:
# Predecimos

preds = xg_reg.predict(X_test)
preds

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [558]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, xg_reg.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.7901477335753542
Test acuracy:  0.7978367241823332
ROC auc score:  0.7032622949126991
Confusion matrix: 
[[2972  700]
 [  85  126]]


In [559]:
# fijandose lo de RFE

In [560]:
# Separamos la variable a predecir

X, y = df_final_prueba.iloc[:,:-1], df_final_prueba.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [561]:
cant_ceros = np.sum(y == 0)
cant_unos = np.sum(y == 1)
cant_ceros, cant_unos

(18434, 980)

In [562]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 18434, 1: 1399})
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

In [563]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [564]:
from sklearn.feature_selection import RFE

# Separamos la variable a predecir

X, y = df_final_prueba.iloc[:,:-1], df_final_prueba.iloc[:,-1]

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

ros = RandomOverSampler(random_state=2, sampling_strategy = {0: 18434, 1: 1399})
X_train_res, y_train_res = ros.fit_sample(X, y)
X_train_res_df = pd.DataFrame(X_train_res)
X_train_res_df.columns = X_test.columns

#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

rfe = RFE(xg_reg, 15)
rfe = rfe.fit(X_train_res_df,y_train_res)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False  True  True  True  True False False False  True  True  True  True
 False  True False False False False False False False False  True False
 False False False False False False False False False False False False
 False False False  True  True False False False False False False False
 False False False  True  True False False False False False False False
 False False False  True False False False False False False False False
 False False]
[50  1  1  1  1 16  8  7  1  1  1  1  3  1 18 34 49 59 43 37 31 39  1 21
 20 14  6 38 45  4  2 40 46 15 53 55 33 29 56  1  1 44 30 58 42 19  9 25
 27 52 17  1  1 35 28 12 32 48 13 23 22 41 36  1 47  5 10 24 11 26 51 54
 57 60]


In [565]:
# Predecimos

preds = rfe.predict(X_test)
preds

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [566]:
# Vemos 
train_accuracy = accuracy_score(y_train_res, rfe.predict(X_train_res_df))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.7551555488327535
Test acuracy:  0.7563739376770539
ROC auc score:  0.6880401449679397
Confusion matrix: 
[[2808  864]
 [  82  129]]


In [567]:
# Separamos la variable a predecir

X, y = labels_con_promedios_num.iloc[:,:-1],labels_con_promedios_num.iloc[:,-1]

In [568]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [569]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [570]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 6, min_child_weight=1)


In [571]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=1)

In [572]:
# Predecimos

preds = xg_reg.predict_proba(X_test)[:, 1]
preds

array([0.36512432, 0.54599017, 0.28901175, ..., 0.42871228, 0.34284964,
       0.34284964], dtype=float32)

In [573]:
# Vemos que onda

train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, xg_reg.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, xg_reg.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9318781791256198
Test acuracy:  0.9296935359258306
ROC auc score:  0.7383659589433501
Confusion matrix: 
[[3576  116]
 [ 157   34]]


In [574]:
# sin duplicar

# Separamos la variable a predecir

X, y = df_final_prueba.iloc[:,:-1], df_final_prueba.iloc[:,-1]

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)
#Entrenamos

xg_reg.fit(X_train,y_train)
# Predecimos

preds = xg_reg.predict(X_test)

# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8779859635567574
Test acuracy:  0.8720061807880505
ROC auc score:  0.6218688370556227
Confusion matrix: 
[[3314  358]
 [ 139   72]]


In [575]:
df_ejemplo = pd.read_csv("trocafone_kaggle_submit_sample_all_0.csv")
df_ejemplo.head()

,person,label
0,4886f805,0
1,0297fc1e,0
2,2d681dd8,0
3,cccea85e,0
4,4c8a8b93,0


In [576]:
df_persons.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [577]:
a_predecir = pd.merge(df_final, df_persons, on="person", how="inner")
a_predecir["person"].count() - df_persons["person"].count()

0

In [578]:
a_predecir_num = a_predecir.drop(columns="person")

In [579]:
# Predecimos

labels_predicciones = xg_reg.predict(a_predecir_num)

personas_a_predecir = df_persons[["person"]]
personas_a_predecir["label"] = pd.Series(labels_predicciones, dtype=int)
personas_a_predecir.shape

(19415, 2)

In [580]:
personas_a_predecir.to_csv("predicciones.csv", encoding='utf-8', index=False)

In [581]:
from sklearn.feature_selection import RFE

# Separamos la variable a predecir

X, y = df_final_prueba.iloc[:,:-1], df_final_prueba.iloc[:,-1]

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)

rfe = RFE(xg_reg, 14)
rfe = rfe.fit(X_train,y_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[False False  True False  True False  True False  True False  True False
 False False False False False False False False False False  True False
 False False False  True False  True False False False  True False False
 False False False  True  True False False False False False False False
 False False False False False  True False False False False False False
 False False False False False  True False False False False  True False
 False False]
[ 2  3  1 15  1 14  1 16  1  4  1  6  9  8 32 61  7 37 24 20 18 48  1 49
 56 44 25  1 12  1 46 22 26  1 54 57 59 60 58  1  1 39 43 41 21 36 17 29
 19 52  5 11 35  1 30 42 38 23 50 47 45 51 27 31 13  1 10 33 28 34  1 40
 53 55]


In [312]:
preds = rfe.predict(X_test)

# Vemos 
train_accuracy = accuracy_score(y_train, rfe.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8710321292898074
Test acuracy:  0.8622199330414628
ROC auc score:  0.6300955611312455
Confusion matrix: 
[[3270  402]
 [ 133   78]]


In [589]:
# Separamos la variable a predecir

X, y = df_final_prueba.iloc[:,:-1],df_final_prueba.iloc[:,-1]

In [590]:
# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)


In [591]:

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test)

In [592]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBClassifier(objective ='binary:logistic', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 6, min_child_weight=1)


In [586]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=6, seed=None,
       silent=True, subsample=1)

In [594]:
# Predecimos

preds = rfe.predict_proba(X_test)[:, 1]
preds

array([0.36007977, 0.313215  , 0.38032264, ..., 0.31013724, 0.32352626,
       0.32553867], dtype=float32)

In [598]:
rfe = RFE(xg_reg, 25)
rfe = rfe.fit(X_train,y_train)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True  True False  True  True  True  True  True  True
 False False  True  True  True False False False False False  True False
 False  True False False False  True False  True False  True False  True
 False False False  True  True False False False False False False False
 False False  True False False False False False False False False False
 False False False False False False  True False  True False False False
 False False]
[ 1  1  1  1  1 11  1  1  1  1  1  1 22  2  1  1  1 30  8 12 43 36  1 13
 25  1 32 26 33  1  5  1 40  1 46  1 50 49 35  1  1 44 28 38 23 42 27  4
 37 39  1 17 31 34  3 20 19 14  7 15  9 10 18 29 21  6  1 24  1 41 16 45
 47 48]


In [599]:
# Vemos que onda
preds = rfe.predict_proba(X_test)[:, 1]
preds
train_accuracy = accuracy_score(y_train, rfe.predict(X_train))
test_accuracy = accuracy_score(y_test, rfe.predict(X_test))
matriz_de_confusion = confusion_matrix(y_test, rfe.predict(X_test))
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9330371515034447
Test acuracy:  0.9242853463816637
ROC auc score:  0.7678602773389502
Confusion matrix: 
[[3553  119]
 [ 175   36]]


---

# Time features de Santi

## Hora

#### Cantidad de eventos por hora. (Todas las horas)

In [262]:
df["day"] = df["timestamp"].dt.day
df["day_of_week"] = df['timestamp'].dt.weekday_name
df['day_of_year'] = df['timestamp'].dt.dayofyear
df['hour'] = df['timestamp'].dt.hour
df['minute'] = df['timestamp'].dt.minute
#df_events['second'] = df_events['timestamp'].dt.second
df['week_of_year'] = df['timestamp'].dt.weekofyear

In [263]:
df_hour = df[['person','hour']]

In [264]:
df_hour1 = pd.concat([df_hour['person'],pd.get_dummies(df_hour['hour'])],axis = 1).groupby('person').sum()\
            .reset_index()
df_hour1.head()

,person,0,1,2,3,4,5,6,7,8,...,14,15,16,17,18,19,20,21,22,23
0,0008ed71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00091926,117.0,122.0,33.0,15.0,26.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,50.0,10.0,0.0,3.0,7.0,65.0
2,00091a7a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,000ba417,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,59.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0
4,000c79fe,17.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [265]:
# Merge

labels_con_horas = pd.merge(df_hour1, labels_con_dia_semana, on="person", how="inner")
labels_con_horas_num = labels_con_horas.drop(columns="person")

### Prueba

In [347]:
# Separamos la variable a predecir

X, y = labels_con_horas_num.iloc[:,:-1], labels_con_horas_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [348]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [349]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [350]:
# Predecimos

preds = xg_reg.predict(X_test)

In [351]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.888640546936629
Test acuracy:  0.8779910596897187
ROC auc score:  0.6152107564848149
Confusion matrix: 
[[3277  334]
 [ 130   62]]


In [352]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.888640546936629
Test acuracy:  0.8779910596897187

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.91      0.93      3611
           1       0.16      0.32      0.21       192

   micro avg       0.88      0.88      0.88      3803
   macro avg       0.56      0.62      0.57      3803
weighted avg       0.92      0.88      0.90      3803


Precision Score:  0.15656565656565657
Recall Score:  0.3229166666666667
F1 Score:  0.2108843537414966
Cohen Kappa Score:  0.1533074228683845
ROC auc score:  0.6152107564848149
Confusion matrix: 
[[3277  334]
 [ 130   62]]


### Prueba duplicando los compradores

In [353]:
# Separamos la variable a predecir

X, y = labels_con_horas_num.iloc[:,:-1], labels_con_horas_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [354]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [396]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [397]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [398]:
# Predecimos

preds = xg_reg.predict(X_test)

In [399]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.7140353073744835
Test acuracy:  0.7125953194846174
ROC auc score:  0.6735921201421582
Confusion matrix: 
[[2589 1022]
 [  71  121]]


In [363]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6731563791160636
Test acuracy:  0.6655272153562977

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.67      0.79      3611
           1       0.09      0.66      0.17       192

   micro avg       0.67      0.67      0.67      3803
   macro avg       0.53      0.66      0.48      3803
weighted avg       0.93      0.67      0.76      3803


Precision Score:  0.0945945945945946
Recall Score:  0.65625
F1 Score:  0.16535433070866143
Cohen Kappa Score:  0.08456547755571042
ROC auc score:  0.6611352464691221
Confusion matrix: 
[[2405 1206]
 [  66  126]]


# PREDICCIÓN...

In [403]:
df_ejemplo = pd.read_csv("trocafone_kaggle_submit_sample_all_0.csv")
df_ejemplo.head()

,person,label
0,4886f805,0
1,0297fc1e,0
2,2d681dd8,0
3,cccea85e,0
4,4c8a8b93,0


In [400]:
df_persons.head()

,person
0,4886f805
1,0297fc1e
2,2d681dd8
3,cccea85e
4,4c8a8b93


In [405]:
labels_con_horas = pd.merge(labels_con_horas, df_persons, on="person", how="inner")
labels_con_horas

,0_x,1_x,2_x,3_x,4_x,5_x,6_x,7,8,9,...,cant_mod,cant_ev,cant_mes_5_x,cant_entradas,cant_mes_5_y,entradas_30_dias,cant_mes,cant_dia_freq,label,person


In [ ]:
# Predecimos

Predicciones = xg_reg.predict(X_test)

#### Cantidad de eventos en fin de semana.

In [279]:
df_day_of_week3 = df[['person','day_of_week']]

In [280]:
df_day_of_week3['fin_de_semana'] = df_day_of_week3['day_of_week'].isin(['Saturday','Sunday'])

C:\Users\seba\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [281]:
df_day_of_week3 = df_day_of_week3.groupby('person').sum().reset_index()

In [282]:
# Merge

labels_con_findes = pd.merge(df_day_of_week3, labels_con_horas, on="person", how="inner")
labels_con_findes_num = labels_con_findes.drop(columns="person")

### Prueba

In [283]:
# Separamos la variable a predecir

X, y = labels_con_findes_num.iloc[:,:-1], labels_con_findes_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [284]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [285]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [286]:
# Predecimos

preds = xg_reg.predict(X_test)

In [287]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.8519589797528268
Test acuracy:  0.8543255324743624

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.88      0.92      3611
           1       0.13      0.34      0.19       192

   micro avg       0.85      0.85      0.85      3803
   macro avg       0.55      0.61      0.56      3803
weighted avg       0.92      0.85      0.88      3803


Precision Score:  0.13360323886639677
Recall Score:  0.34375
F1 Score:  0.1924198250728863
Cohen Kappa Score:  0.12909428967551584
ROC auc score:  0.6126116380504015
Confusion matrix: 
[[3183  428]
 [ 126   66]]


### Prueba duplicando los compradores

In [288]:
# Separamos la variable a predecir

X, y = labels_con_findes_num.iloc[:,:-1], labels_con_findes_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [289]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [290]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [291]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [292]:
# Predecimos

preds = xg_reg.predict(X_test)

In [293]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6854263177663703
Test acuracy:  0.6831448856166185

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.68      0.80      3611
           1       0.10      0.65      0.17       192

   micro avg       0.68      0.68      0.68      3803
   macro avg       0.54      0.67      0.49      3803
weighted avg       0.93      0.68      0.77      3803


Precision Score:  0.09897070467141726
Recall Score:  0.6510416666666666
F1 Score:  0.17182130584192443
Cohen Kappa Score:  0.09225903078080788
ROC auc score:  0.6679467541308963
Confusion matrix: 
[[2473 1138]
 [  67  125]]


In [296]:
labels_con_findes_predecir["person"].count()

0

In [295]:
labels_con_findes_predecir = pd.merge(labels_con_findes, df_persons, on="person", how="inner")
labels_con_findes_predecir["person"].count() - df_persons["person"].count()

-19415

#### Cantidad de eventos en rango de horas (Morning , Afternoon, Evening y Night) 

Morning     5 a 12  

Afternoon     12 a 17

Evening     17 a 21

Night         21 a 4 

In [319]:
df_hour_rango2 = df_hour.copy()

In [320]:
df_hour_rango2['Morning'] = (df_hour_rango2['hour'].isin(list(range(5,13)))).astype(int)
df_hour_rango2['Afternoon'] = (df_hour_rango2['hour'].isin(list(range(13,18)))).astype(int)
df_hour_rango2['Evening'] = (df_hour_rango2['hour'].isin(list(range(18,21)))).astype(int)
df_hour_rango2['Night'] = (df_hour_rango2['hour'].isin([22,23,0,1,2,3,4])).astype(int)
df_hour_rango2.drop(columns = ['hour'],inplace = True)

In [321]:
df_hour_rango2a = df_hour_rango2.groupby('person').sum().reset_index()

#### Visita la pagina en rango de horas (Morning , Afternoon, Evening y Night) 

In [322]:
df_hour_rango2b = df_hour_rango2.drop_duplicates().groupby('person').sum().reset_index()

In [323]:
# Merge

labels_con_rango_horario = pd.merge(df_hour_rango2b, labels_con_findes, on="person", how="inner")
labels_con_rango_horario_num = labels_con_rango_horario.drop(columns="person")

### Prueba

In [324]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [325]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [326]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [327]:
# Predecimos

preds = xg_reg.predict(X_test)

In [328]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.850512753089666
Test acuracy:  0.8535366815671838

Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.88      0.92      3611
           1       0.13      0.34      0.19       192

   micro avg       0.85      0.85      0.85      3803
   macro avg       0.55      0.61      0.56      3803
weighted avg       0.92      0.85      0.88      3803


Precision Score:  0.13279678068410464
Recall Score:  0.34375
F1 Score:  0.19158200290275762
Cohen Kappa Score:  0.1280750665076711
ROC auc score:  0.6121962406535586
Confusion matrix: 
[[3180  431]
 [ 126   66]]


### Prueba duplicando los compradores

In [329]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [330]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [331]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [332]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [333]:
# Predecimos

preds = xg_reg.predict(X_test)

In [334]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6285839489169901
Test acuracy:  0.6108335524585853

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.60      0.75      3611
           1       0.09      0.72      0.16       192

   micro avg       0.61      0.61      0.61      3803
   macro avg       0.53      0.66      0.45      3803
weighted avg       0.93      0.61      0.72      3803


Precision Score:  0.08876117496807152
Recall Score:  0.7239583333333334
F1 Score:  0.15813424345847552
Cohen Kappa Score:  0.07492854595329301
ROC auc score:  0.664388471568356
Confusion matrix: 
[[2184 1427]
 [  53  139]]


## Con los datos transformados

In [335]:
x = labels_con_rango_horario.loc[:,labels_con_rango_horario.columns != 'person']
maxAbsScaler = preprocessing.MaxAbsScaler()
x_scaled = maxAbsScaler.fit_transform(x)
labels_con_rango_horario = pd.concat([labels_con_rango_horario['person'],pd.DataFrame(x_scaled)],axis = 1)
labels_con_rango_horario_num = labels_con_rango_horario.drop(columns="person")

### Prueba

In [336]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [337]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [338]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [339]:
# Predecimos

preds = xg_reg.predict(X_test)

In [340]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.850512753089666
Test acuracy:  0.8535366815671838

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.88      0.92      3611
         1.0       0.13      0.34      0.19       192

   micro avg       0.85      0.85      0.85      3803
   macro avg       0.55      0.61      0.56      3803
weighted avg       0.92      0.85      0.88      3803


Precision Score:  0.13279678068410464
Recall Score:  0.34375
F1 Score:  0.19158200290275762
Cohen Kappa Score:  0.1280750665076711
ROC auc score:  0.6121962406535586
Confusion matrix: 
[[3180  431]
 [ 126   66]]


### Prueba duplicando los compradores

In [341]:
# Separamos la variable a predecir

X, y = labels_con_rango_horario_num.iloc[:,:-1], labels_con_rango_horario_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [342]:
# Duplico los compradores
y_train_df = pd.DataFrame(y_train)
y_train_df.columns = ["label"]
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [343]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [344]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [345]:
# Predecimos

preds = xg_reg.predict(X_test)

In [346]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.6285839489169901
Test acuracy:  0.6108335524585853

Classification Report:
              precision    recall  f1-score   support

         0.0       0.98      0.60      0.75      3611
         1.0       0.09      0.72      0.16       192

   micro avg       0.61      0.61      0.61      3803
   macro avg       0.53      0.66      0.45      3803
weighted avg       0.93      0.61      0.72      3803


Precision Score:  0.08876117496807152
Recall Score:  0.7239583333333334
F1 Score:  0.15813424345847552
Cohen Kappa Score:  0.07492854595329301
ROC auc score:  0.664388471568356
Confusion matrix: 
[[2184 1427]
 [  53  139]]


# -------- Borrador -----------------

In [ ]:
xgb.plot_importance(xg_reg)
plt.rcParams['figure.figsize'] = [10, 10]
plt.show()

objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc"

In [ ]:
params = {"objective":"binary:hinge",'colsample_bytree': 0.8,
          'learning_rate': 0.1, 'max_depth': 5, "n_estimators" : 6}

cv_results = xgb.cv(dtrain=data_dmatrix, params=params, nfold=10,
                    num_boost_round=10,
                    metrics="auc", as_pandas=True, seed=123);

In [ ]:
#cv_results.sort_values("train-auc-mean", ascending=False)
cv_results

In [ ]:
tf_test = pd.read_csv("trocafone_kaggle_test.csv")
tf_test.head()

## Cantidad de entradas desde campañas

In [397]:
df_events_campaign = df[['person','campaign_source']].dropna()


top_5_campanias = df_events_campaign['campaign_source'].value_counts().head(5).index
df_events_campaign = df_events_campaign.loc[df_events_campaign['campaign_source'].isin(top_5_campanias)]
df_events_campaign = pd.concat([df_events_campaign['person'],\
        pd.get_dummies(df_events_campaign['campaign_source'])],axis = 1).groupby('person').sum().reset_index()

df_events_campaign = pd.merge(df_events_campaign, todas_las_personas, on="person", how="right").fillna(0)


df_events_campaign.head()

,person,criteo,emblue,google,rtbhouse,zanox
0,00091926,1.0,0.0,13.0,1.0,0.0
1,00091a7a,0.0,0.0,1.0,0.0,0.0
2,000ba417,0.0,0.0,1.0,0.0,0.0
3,000c79fe,0.0,0.0,1.0,0.0,0.0
4,000e4d9e,0.0,0.0,4.0,0.0,9.0


In [398]:
# Veo cuantas personas a predecir pierdo
df_events_campaign_predecir = pd.merge(df_events_campaign, df_persons, on="person", how="inner")
df_events_campaign_predecir["person"].count() - df_persons["person"].count()

0

In [405]:
# Merge
labels_con_camps = pd.merge(df_events_campaign, labels_con_dia_semana, on="person", how="inner")
labels_con_camps_num = labels_con_camps.drop(columns="person")
labels_con_camps_num.head()


,criteo,emblue,google,rtbhouse,zanox,0,1,2,3,4,...,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1,4,9,1,17,17,16,17,17,0
1,5.0,0.0,17.0,7.0,0.0,3.0,6.0,2.0,2.0,3.0,...,12,63,189,15,471,290,358,290,72,0
2,3.0,0.0,4.0,0.0,0.0,0.0,3.0,6.0,3.0,0.0,...,6,20,52,1,96,54,64,54,17,0
3,2.0,0.0,10.0,0.0,0.0,0.0,1.0,0.0,2.0,0.0,...,10,8,17,3,54,5,26,46,22,0
4,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,1,3,17,1,38,26,25,26,26,0


### Prueba

In [406]:
# Separamos la variable a predecir

X, y = labels_con_camps_num.iloc[:,:-1], labels_con_camps_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [407]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1000)


In [408]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1000, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [409]:
# Predecimos

preds = xg_reg.predict(X_test)

In [410]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8869953904176561
Test acuracy:  0.8748603351955307
ROC auc score:  0.5785714285714286
Confusion matrix: 
[[1548  172]
 [  52   18]]


## Cantidad de visitas a productos Apple y Samsung

In [365]:
df_productos = df[["person", "event", "model"]]
df_productos = df_productos.loc[df_productos["event"] == "viewed product"]
df_productos = df_productos.loc[(df_productos["model"].str.contains("iPhone") )|( df_productos["model"].str.contains("Samsung") )]
df_productos.head()

df_apple = df_productos.loc[df_productos["model"].str.contains("iPhone")]
df_samsung = df_productos.loc[df_productos["model"].str.contains("Samsung")]                           

vistas_apple = df_apple["person"].value_counts().reset_index()
vistas_apple.columns = ["person", "vistas_apple"]

vistas_samsung = df_apple["person"].value_counts().reset_index()
vistas_samsung.columns = ["person", "vistas_samsung"]

vistas_prods = pd.merge(vistas_samsung, vistas_apple, on="person", how="inner")

vistas_prods = pd.merge(vistas_prods, todas_las_personas, on="person", how="right").fillna(0)

vistas_prods.head()

,person,vistas_samsung,vistas_apple
0,6abd2bf1,2051.0,2051.0
1,b1f4dbf6,1866.0,1866.0
2,9bf968c5,1557.0,1557.0
3,bfb74b38,1526.0,1526.0
4,6f19cfd9,1442.0,1442.0


In [366]:
# Veo cuantas personas a predecir pierdo
vistas_prods_predecir = pd.merge(vistas_prods, df_persons, on="person", how="inner")
vistas_prods_predecir["person"].count() - df_persons["person"].count()

0

In [375]:
# Merge

df_con_vistas_prods = pd.merge(vistas_prods,labels_con_dia_semana, on="person", how="inner")

df_con_vistas_prods_num = df_con_vistas_prods.drop(columns="person")

### Prueba

In [376]:
# Separamos la variable a predecir

X, y = df_con_vistas_prods_num.iloc[:,:-1], df_con_vistas_prods_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [377]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6)


In [378]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [379]:
# Predecimos

preds = xg_reg.predict(X_test)

In [380]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.9113004609582344
Test acuracy:  0.9016759776536313
ROC auc score:  0.582583419606594
Confusion matrix: 
[[1600  132]
 [  44   14]]


## Cantidad de busquedas en 'searched products' y 'search engine hit'

In [383]:
df_search = df.loc[(df["event"] == "searched products" )| (df["event"] == "search engine hit" )]

searched_products = df_search.loc[df_search["event"] == "searched products"]
cant_searched_products = searched_products["person"].value_counts().reset_index()
cant_searched_products.columns = ["person", "cant_search_prod"]

engine_hit = df_search.loc[df_search["event"] == "search engine hit"]
cant_engine_hit = engine_hit["person"].value_counts().reset_index()
cant_engine_hit.columns = ["person", "cant_engine_hit"]

cant_busquedas = pd.merge(cant_engine_hit, cant_searched_products, on="person", how="inner")

cant_busquedas = pd.merge(cant_busquedas, todas_las_personas, on="person", how="right").fillna(0)



In [384]:
# Veo cuantas personas a predecir pierdo
cant_busquedas_predecir = pd.merge(cant_busquedas, df_persons, on="person", how="inner")
cant_busquedas_predecir["person"].count() - df_persons["person"].count()

0

In [391]:
# Merge

labels_con_busq = pd.merge(cant_busquedas, labels_con_dia_semana, on="person", how="inner")
labels_con_busq_num = labels_con_busq.drop(columns="person")
labels_con_busq_num.head()

,cant_engine_hit,cant_search_prod,0,1,2,3,4,5,6,Chrome 65.0,...,cant_eng,cant_mod,cant_ev,check_tot,cant_entradas,cant_mes_5,entradas_30_dias,cant_mes,cant_dia_freq,label
0,245.0,3.0,22.0,15.0,22.0,16.0,28.0,19.0,19.0,4.0,...,245,170,744,47,2006,388,378,861,198,0
1,136.0,739.0,9.0,8.0,14.0,13.0,6.0,10.0,6.0,0.0,...,136,61,739,10,1609,1609,1608,1609,191,0
2,130.0,214.0,18.0,14.0,12.0,14.0,7.0,9.0,12.0,0.0,...,130,87,214,4,855,760,759,760,138,0
3,102.0,2.0,17.0,21.0,24.0,24.0,21.0,14.0,14.0,29.0,...,102,130,450,2,1025,103,105,585,102,0
4,93.0,121.0,26.0,23.0,33.0,20.0,33.0,23.0,7.0,56.0,...,93,416,692,17,1593,412,420,956,216,0


### Prueba

In [392]:
# Separamos la variable a predecir

X, y = labels_con_busq_num.iloc[:,:-1], labels_con_busq_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y)

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [393]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, scale_pos_weight = 100, min_child_weight=1)


In [394]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=5, min_child_weight=1000, missing=None, n_estimators=6,
       n_jobs=1, nthread=None, objective='binary:hinge', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=100, seed=None,
       silent=True, subsample=1)

In [395]:
# Predecimos

preds = xg_reg.predict(X_test)

In [396]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.8112864925268892
Test acuracy:  0.8011173184357542
ROC auc score:  0.6384819951317123
Confusion matrix: 
[[1403  320]
 [  36   31]]


### duplico compradores

In [1022]:
# Separamos la variable a predecir

X, y = labels_con_busq_num.iloc[:,:-1], labels_con_busq_num.iloc[:,-1]

# Convertimos los datos a DMatrix

data_dmatrix = xgb.DMatrix(data=X,label=y, weight={1: 1, 0: 0.1})

#Creamos set de entrenamiento y test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [1023]:
# Duplico los compradores

y_train_df = y_train.to_frame()
X_train_df = X_train.reset_index()
train = pd.merge(X_train, y_train_df, how="inner", left_index=True, right_index=True)
train_compradores = train.loc[train["label"] == 1]
train_dup = pd.concat([train, train_compradores], ignore_index=True)
X_train, y_train = train_dup.drop(columns="label"), train_dup["label"]

In [1024]:
#Instanciamos el regresor de XGBoost

ratio = float(np.sum(y_train == 0)) / np.sum(y_train == 1)

xg_reg = xgb.XGBRegressor(objective ='binary:hinge', 
                colsample_bytree = 0.8, learning_rate = 0.1,
                max_depth = 5, n_estimators = 6, eval_metric = "auc", min_child_weight=1)


In [1025]:
#Entrenamos

xg_reg.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.8, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=5, min_child_weight=1, missing=None,
       n_estimators=6, n_jobs=1, nthread=None, objective='binary:hinge',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=None, silent=True, subsample=1)

In [1026]:
# Predecimos

preds = xg_reg.predict(X_test)

In [1027]:
# Vemos 
train_accuracy = accuracy_score(y_train, xg_reg.predict(X_train))
test_accuracy = accuracy_score(y_test, preds)
matriz_de_confusion = confusion_matrix(y_test, preds)
area_debajo_de_curva = roc_auc_score(y_test, preds)

print("Train accuracy: ", train_accuracy)
print("Test acuracy: ", test_accuracy)
print("ROC auc score: ", area_debajo_de_curva)
print("Confusion matrix: ")
print(matriz_de_confusion)

Train accuracy:  0.7886639676113361
Test acuracy:  0.764804469273743
ROC auc score:  0.6339645359967429
Confusion matrix: 
[[1336  387]
 [  34   33]]


In [1028]:
from sklearn.metrics import accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,\
                            f1_score,precision_score,recall_score,classification_report,cohen_kappa_score
# Pruebo todas las metricas.
# accuracy_score, roc_auc_score, mean_squared_error, confusion_matrix,
# f1_score,precision_score,recall_score,classification_report,cohen_kappa_score

trainAccuracy = accuracy_score(y_train, xg_reg.predict(X_train))
testAccuracy = accuracy_score(y_test, preds)

meanSquaredError = mean_squared_error(y_test, preds)
classificationReport = classification_report(y_test, preds)
precisionScore = precision_score(y_test, preds)
recallScore = recall_score(y_test, preds)
f1Score = f1_score(y_test, preds)
cohenKappaScore = cohen_kappa_score(y_test, preds)
areaDebajoDeCurva = roc_auc_score(y_test, preds)
matrizDeConfusion = confusion_matrix(y_test, preds)

print("Train accuracy: ", trainAccuracy)
print("Test acuracy: ", testAccuracy)
print()
print("Classification Report:")
print(classificationReport)
print()
print("Precision Score: ",precisionScore)
print("Recall Score: ",recallScore)
print("F1 Score: ",f1Score)
print("Cohen Kappa Score: ",cohenKappaScore)
print("ROC auc score: ", areaDebajoDeCurva)
print("Confusion matrix: ")
print(matrizDeConfusion)

Train accuracy:  0.7886639676113361
Test acuracy:  0.764804469273743

Classification Report:
              precision    recall  f1-score   support

           0       0.98      0.78      0.86      1723
           1       0.08      0.49      0.14        67

   micro avg       0.76      0.76      0.76      1790
   macro avg       0.53      0.63      0.50      1790
weighted avg       0.94      0.76      0.84      1790


Precision Score:  0.07857142857142857
Recall Score:  0.4925373134328358
F1 Score:  0.135523613963039
Cohen Kappa Score:  0.07585995462628004
ROC auc score:  0.6339645359967429
Confusion matrix: 
[[1336  387]
 [  34   33]]
